# データサイエンス100本ノック（構造化データ加工編） - Python
# for Google Colab

## 【注意】オリジナル版との変更点
1. Google Colab ではDockerを使えないので、2020.06.18時点の 100knocks-preprocess/docker/work/data にあるCSVファイルをPostgreSQLから入手する代わりに使います。
2. オリジナルのCSVデータ, geocode.csvの'latitude'列名の初めにスペースが入っていたため、それを削除しました。

   オリジナル(100knocks-preprocess ver.1.0): ' latitude' --> 'latitude'
   

4. オリジナルの解答を元に、必要のないlibraryはimportせず、そして必要なlibraryをAzureNotebookでインストールするように最初のセルを改変してあります。
5. また、SQLではなく上記CSVからデータを読み込むように'はじめに'の最初のセルを改変してあります。

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートと~~データベース（PostgreSQL）~~ 100knocks-preprocess/docker/work/data にあるCSVファイルからのデータ読み込みを行います。geocode.csvに変更を加えたため、またgit cloneをするとColabの簡便さを損なうため、noguhiro2002のgithubレポジトリから直接読み込みます。
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能)
- オリジナルの解答例を元に、必要なライブラリーをpipでインストールします。
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


# データを github/noguhiro2002/100knocks-preprocess/work/data フォルダよりDataframe形式でRead
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_category = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/category.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')
df_store = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/store.csv')
df_geocode = pd.read_csv('https://raw.githubusercontent.com/noguhiro2002/100knocks-preprocess_ForColab-AzureNotebook/master/data/geocode.csv')

/Users/yuya/.local/share/virtualenvs/test36--SgIQ7cC/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
df_receipt.head(10)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1257206400   S14006         112               1   
1   20181118   1258502400   S13008        1132               2   
2   20170712   1215820800   S14028        1102               1   
3   20190205   1265328000   S14042        1132               1   
4   20180821   1250812800   S14025        1102               2   
5   20190605   1275696000   S13003        1112               1   
6   20181205   1259971200   S14024        1102               2   
7   20190922   1285113600   S14040        1102               1   
8   20170504   1209859200   S13020        1112               2   
9   20191010   1286668800   S14027        1102               1   

      customer_id  product_cd  quantity  amount  
0  CS006214000001  P070305012         1     158  
1  CS008415000097  P070701017         1      81  
2  CS028414000014  P060101005         1     170  
3  ZZ000000000000  P050301001         1      25  
4  CS025415000050  P060102007         1      90  
5  CS003515000195  P050102002         1     138  
6  CS024514000042  P080101005         1      30  
7  CS040415000178  P070501004         1     128  
8  ZZ000000000000  P071302010         1     770  
9  CS027514000015  P071101003         1     680

---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [6]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].head(10)
# カラム名はクォーテーションで囲むのを忘れがち注意
# 複数カラム選択の際、リストとして鍵かっこに入れる＝見た目上二重鍵カッコになる

sales_ymd     customer_id  product_cd  amount
0   20181103  CS006214000001  P070305012     158
1   20181118  CS008415000097  P070701017      81
2   20170712  CS028414000014  P060101005     170
3   20190205  ZZ000000000000  P050301001      25
4   20180821  CS025415000050  P060102007      90
5   20190605  CS003515000195  P050102002     138
6   20181205  CS024514000042  P080101005      30
7   20190922  CS040415000178  P070501004     128
8   20170504  ZZ000000000000  P071302010     770
9   20191010  CS027514000015  P071101003     680

---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [10]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].rename(columns={'sales_ymd':'sales_date'}).head(10)
# renameの引数columns=には辞書型での記述

sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680

In [11]:
df_receipt.rename(columns={'sales_ymd':'sales_date'})[['sales_date','customer_id','product_cd','amount']].head(10)
# 上の違う書き方。順番が違うが、この場合はカラム名変更後の取り出しなので変更後のカラム名で取り出す。

sales_date     customer_id  product_cd  amount
0    20181103  CS006214000001  P070305012     158
1    20181118  CS008415000097  P070701017      81
2    20170712  CS028414000014  P060101005     170
3    20190205  ZZ000000000000  P050301001      25
4    20180821  CS025415000050  P060102007      90
5    20190605  CS003515000195  P050102002     138
6    20181205  CS024514000042  P080101005      30
7    20190922  CS040415000178  P070501004     128
8    20170504  ZZ000000000000  P071302010     770
9    20191010  CS027514000015  P071101003     680

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [16]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query("customer_id=='CS018205000001'").head(10)
# queryの書き方：カラム名は変数として定義されていないためクォーテーションが必要なのは同じだが、
# クォーテーション内では変数のように振舞うため検索ワード側に別のクォーテーションが入る

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

In [214]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']][df_receipt.customer_id=='CS018205000001'].head(10)
# 別の書き方。queryメソッドを使わず条件抽出。テーブル名.カラム名で指定可能。

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [8]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].query('customer_id=="CS018205000001" and amount>=1000')
# クォーテーション内でandで繋げる

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

In [219]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
[(df_receipt.customer_id=="CS018205000001") & (df_receipt.amount>=1000)]
# 違う書き方。各条件式はカッコで囲む必要がある。また、繋ぎは'and'では不可で'&'である必要がある。

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100

---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [220]:
df_receipt[['sales_ymd','customer_id','product_cd','quantity','amount']]\
.query('customer_id=="CS018205000001" and (amount>=1000 or quantity>=5)')
# クォーテーション内でカッコでまとめられる

sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100

In [222]:
df_receipt[['sales_ymd','customer_id','product_cd','quantity','amount']]\
[(df_receipt.customer_id=="CS018205000001") & ((df_receipt.amount>=1000) | (df_receipt.quantity>=5))]
# 違う書き方。同様にカッコでまとめられる。こちらの場合はカッコが二重っぽくなる。

sales_ymd     customer_id  product_cd  quantity  amount
36      20180911  CS018205000001  P071401012         1    2200
9843    20180414  CS018205000001  P060104007         6     600
21110   20170614  CS018205000001  P050206001         5     990
68117   20190226  CS018205000001  P071401020         1    2200
72254   20180911  CS018205000001  P071401005         1    1100

---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [223]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
.query('customer_id=="CS018205000001" and 2000>=amount>=1000')

sales_ymd     customer_id  product_cd  amount
72254   20180911  CS018205000001  P071401005    1100

In [226]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
[(df_receipt.customer_id=="CS018205000001") & (2000>=df_receipt.amount>=1000)]
# この書き方では数字で挟むことができないので、&をもう一つ増やして分ける必要がありそう。この辺になるとqueryメソッドの方が柔軟性が上がってくる。

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [229]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
.query('customer_id=="CS018205000001" and product_cd != "P071401019"')

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

In [230]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
[(df_receipt.customer_id=="CS018205000001") & (df_receipt.product_cd != "P071401019")]
# 違う書き方

sales_ymd     customer_id  product_cd  amount
36      20180911  CS018205000001  P071401012    2200
9843    20180414  CS018205000001  P060104007     600
21110   20170614  CS018205000001  P050206001     990
27673   20170614  CS018205000001  P060702015     108
27840   20190216  CS018205000001  P071005024     102
28757   20180414  CS018205000001  P071101002     278
39256   20190226  CS018205000001  P070902035     168
58121   20190924  CS018205000001  P060805001     495
68117   20190226  CS018205000001  P071401020    2200
72254   20180911  CS018205000001  P071401005    1100
88508   20190216  CS018205000001  P040101002     218
91525   20190924  CS018205000001  P091503001     280

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [13]:
df_store.query('prefecture_cd!="13" and floor_area<=900')

store_cd store_name  prefecture_cd prefecture            address  \
18   S14046       北山田店             14       神奈川県   神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店             14       神奈川県  神奈川県横浜市港北区日吉本町四丁目   
38   S12013       習志野店             12        千葉県       千葉県習志野市芝園一丁目   

                      address_kana        tel_no  longitude  latitude  \
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
38             チバケンナラシノシシバゾノイッチョウメ  047-123-4002   140.0220  35.66122   

    floor_area  
18       831.0  
20       890.0  
38       808.0

---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [15]:
df_store.query('store_cd.str.startswith("S14")',engine='python').head(10)
# numexprがインストールされている場合、引数engine='python'を指定する必要がある

store_cd store_name  prefecture_cd prefecture              address  \
2    S14010        菊名店             14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店             14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店             14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店             14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店             14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店             14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店             14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店             14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   

    floor_area  
2       1732.0  
3       1495.0  
4       1679.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
19      1838.0  
20       890.0

In [231]:
df_store[df_store.store_cd.str.startswith("S14")].head(10)
# 同等：df_store.query('store_cd.str.startswith("S14")',engine='python').head(10)

store_cd store_name  prefecture_cd prefecture              address  \
2    S14010        菊名店             14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店             14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店             14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店             14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店             14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店             14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店             14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店             14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   

    floor_area  
2       1732.0  
3       1495.0  
4       1679.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
19      1838.0  
20       890.0

---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [233]:
df_customer.query('customer_id.str.endswith("1")').head(10)
# 同等
df_customer[df_customer.customer_id.str.endswith('1')].head(10)

customer_id customer_name  gender_cd gender   birth_day  age  \
1    CS037613000071         六角 雅彦          9     不明  1952-04-01   66   
3    CS028811000001        堀井 かおり          1     女性  1933-03-27   86   
14   CS040412000191         川井 郁恵          1     女性  1977-01-05   42   
31   CS028314000011        小菅 あおい          1     女性  1983-11-26   35   
56   CS039212000051        藤島 恵梨香          1     女性  1997-02-03   22   
59   CS015412000111         松居 奈月          1     女性  1972-10-04   46   
63   CS004702000041          野島 洋          0     男性  1943-08-24   75   
74   CS041515000001         栗田 千夏          1     女性  1967-01-02   52   
85   CS029313000221        北条 ひかり          1     女性  1987-06-19   31   
102  CS034312000071         望月 奈央          1     女性  1980-09-20   38   

    postal_cd                  address application_store_cd  application_date  \
1    136-0076       東京都江東区南砂**********               S13037          20150414   
3    245-0016   神奈川県横浜市泉区和泉町**********               S14028          20160115   
14   226-0021  神奈川県横浜市緑区北八朔町**********               S14040          20151101   
31   246-0038   神奈川県横浜市瀬谷区宮沢**********               S14028          20151123   
56   166-0001     東京都杉並区阿佐谷北**********               S13039          20171121   
59   136-0071       東京都江東区亀戸**********               S13015          20150629   
63   176-0022       東京都練馬区向山**********               S13004          20170218   
74   206-0001       東京都多摩市和田**********               S13041          20160422   
85   279-0011       千葉県浦安市美浜**********               S12029          20180810   
102  213-0026   神奈川県川崎市高津区久末**********               S14034          20160106   

        status_cd  
1    0-00000000-0  
3    0-00000000-0  
14   1-20091025-4  
31   1-20080426-5  
56   1-20100215-4  
59   0-00000000-0  
63   0-00000000-0  
74   E-20100803-F  
85   0-00000000-0  
102  0-00000000-0

---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [234]:
df_store.query('address.str.contains("横浜市")')
# 同等
df_store[df_store.address.str.contains('横浜市')]

store_cd store_name  prefecture_cd prefecture              address  \
2    S14010        菊名店             14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
7    S14040       長津田店             14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
12   S14028       二ツ橋店             14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店             14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店             14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
20   S14011      日吉本町店             14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
26   S14048      中川中央店             14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
40   S14042       新山下店             14       神奈川県      神奈川県横浜市中区新山下二丁目   
52   S14006       葛が谷店             14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
26   カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ  045-123-4051   139.5758  35.54912   
40       カナガワケンヨコハマシナカクシンヤマシタニチョウメ  045-123-4047   139.6593  35.43894   
52             カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633  35.53573   

    floor_area  
2       1732.0  
3       1495.0  
7       1548.0  
9       1830.0  
12      1574.0  
16      1341.0  
18       831.0  
20       890.0  
26      1657.0  
40      1044.0  
52      1886.0

---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [235]:
df_customer.query('status_cd.str.contains("^[A-F]")').head(10)
# 方法１：str.containsの場合はデフォルトで正規表現が適用される。
# 問題は〜で始まる、だがstartswithは正規表現が使えないので"^"を先頭に置いてstartswithと同じ意味にする。
# 正規表現ではなく文字列なのに正規表現として取られてしまう場合は第二引数以降に”regex=False"を入れる（デフォルトでTrueになっている）
# str.contains以外に、完全に正規表現で表すstr.match()がある。

#同等
df_customer[df_customer.status_cd.str.contains('^[A-F]')].head(10)

customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053   
6   CS015414000103         奥野 陽子          1     女性  1977-08-09   41  136-0073   
12  CS011215000048         芦田 沙耶          1     女性  1992-02-01   27  223-0062   
15  CS029415000023         梅田 里穂          1     女性  1976-01-17   43  279-0043   
21  CS035415000029         寺沢 真希          9     不明  1977-09-27   41  158-0096   
32  CS031415000106        宇野 由美子          1     女性  1970-02-26   49  151-0053   
33  CS029215000025         石倉 美帆          1     女性  1993-09-28   25  279-0022   
40  CS033605000005         猪股 雄太          0     男性  1955-12-05   63  246-0031   
44  CS033415000229        板垣 菜々美          1     女性  1977-11-07   41  246-0021   
53  CS008415000145         黒谷 麻緒          1     女性  1977-06-27   41  157-0067   

                     address application_store_cd  application_date  \
2        東京都渋谷区代々木**********               S13031          20150529   
6         東京都江東区北砂**********               S13015          20150722   
12  神奈川県横浜市港北区日吉本町**********               S14011          20150228   
15       千葉県浦安市富士見**********               S12029          20150610   
21      東京都世田谷区玉川台**********               S13035          20141220   
32       東京都渋谷区代々木**********               S13031          20150201   
33        千葉県浦安市今川**********               S12029          20150708   
40    神奈川県横浜市瀬谷区瀬谷**********               S14033          20150425   
44  神奈川県横浜市瀬谷区二ツ橋町**********               S14033          20150712   
53      東京都世田谷区喜多見**********               S13008          20150829   

       status_cd  
2   D-20100325-C  
6   B-20100609-B  
12  C-20100421-9  
15  D-20100918-E  
21  F-20101029-F  
32  F-20100511-E  
33  B-20100820-C  
40  F-20100917-E  
44  F-20100326-E  
53  F-20100622-F

In [238]:
df_customer.query('status_cd.str.match("^[A-F]")').head(10)
# str.matchでも同じ結果になる。

#同等
df_customer[df_customer.status_cd.str.match('^[A-F]')].head(10)

customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
2   CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053   
6   CS015414000103         奥野 陽子          1     女性  1977-08-09   41  136-0073   
12  CS011215000048         芦田 沙耶          1     女性  1992-02-01   27  223-0062   
15  CS029415000023         梅田 里穂          1     女性  1976-01-17   43  279-0043   
21  CS035415000029         寺沢 真希          9     不明  1977-09-27   41  158-0096   
32  CS031415000106        宇野 由美子          1     女性  1970-02-26   49  151-0053   
33  CS029215000025         石倉 美帆          1     女性  1993-09-28   25  279-0022   
40  CS033605000005         猪股 雄太          0     男性  1955-12-05   63  246-0031   
44  CS033415000229        板垣 菜々美          1     女性  1977-11-07   41  246-0021   
53  CS008415000145         黒谷 麻緒          1     女性  1977-06-27   41  157-0067   

                     address application_store_cd  application_date  \
2        東京都渋谷区代々木**********               S13031          20150529   
6         東京都江東区北砂**********               S13015          20150722   
12  神奈川県横浜市港北区日吉本町**********               S14011          20150228   
15       千葉県浦安市富士見**********               S12029          20150610   
21      東京都世田谷区玉川台**********               S13035          20141220   
32       東京都渋谷区代々木**********               S13031          20150201   
33        千葉県浦安市今川**********               S12029          20150708   
40    神奈川県横浜市瀬谷区瀬谷**********               S14033          20150425   
44  神奈川県横浜市瀬谷区二ツ橋町**********               S14033          20150712   
53      東京都世田谷区喜多見**********               S13008          20150829   

       status_cd  
2   D-20100325-C  
6   B-20100609-B  
12  C-20100421-9  
15  D-20100918-E  
21  F-20101029-F  
32  F-20100511-E  
33  B-20100820-C  
40  F-20100917-E  
44  F-20100326-E  
53  F-20100622-F

---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [239]:
df_customer.query('status_cd.str.contains("[1-9]$")').head(10)
# 同等
df_customer[df_customer.status_cd.str.contains('[1-9]$')].head(10)

customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
4   CS001215000145         田崎 美紀          1     女性  1995-03-29   24  144-0055   
9   CS033513000180          安斎 遥          1     女性  1962-07-11   56  241-0823   
12  CS011215000048         芦田 沙耶          1     女性  1992-02-01   27  223-0062   
14  CS040412000191         川井 郁恵          1     女性  1977-01-05   42  226-0021   
16  CS009315000023         皆川 文世          1     女性  1980-04-15   38  154-0012   
22  CS015315000033        福士 璃奈子          1     女性  1983-03-17   36  135-0043   
23  CS023513000066         神戸 そら          1     女性  1961-12-17   57  210-0005   
24  CS035513000134         市川 美帆          1     女性  1960-03-27   59  156-0053   
27  CS001515000263         高松 夏空          1     女性  1962-11-09   56  144-0051   
28  CS040314000027       鶴田 きみまろ          9     不明  1986-03-26   33  226-0027   

                     address application_store_cd  application_date  \
4        東京都大田区仲六郷**********               S13001          20170605   
9     神奈川県横浜市旭区善部町**********               S14033          20150728   
12  神奈川県横浜市港北区日吉本町**********               S14011          20150228   
14   神奈川県横浜市緑区北八朔町**********               S14040          20151101   
16       東京都世田谷区駒沢**********               S13009          20150319   
22        東京都江東区塩浜**********               S13015          20141024   
23   神奈川県川崎市川崎区東田町**********               S14023          20150915   
24        東京都世田谷区桜**********               S13035          20150227   
27       東京都大田区西蒲田**********               S13001          20160812   
28    神奈川県横浜市緑区長津田**********               S14040          20150122   

       status_cd  
4   6-20090929-2  
9   6-20080506-5  
12  C-20100421-9  
14  1-20091025-4  
16  5-20080322-1  
22  4-20080219-3  
23  5-20100524-9  
24  8-20100711-9  
27  1-20100804-1  
28  2-20080426-4

---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [240]:
df_customer.query('status_cd.str.contains("^[A-F].*[1-9]$")').head(10)
# 正規表現：^ で始まる　$ で終わる　. 任意の１文字　* 直前の文字の繰り返し　.*で任意の文字の繰り返しなので中間は何でもOKという意味になる。

# 同等
df_customer[df_customer.status_cd.str.contains('^[A-F].*[1-9]$')].head(10)

customer_id customer_name  gender_cd gender   birth_day  age  \
12   CS011215000048         芦田 沙耶          1     女性  1992-02-01   27   
68   CS022513000105        島村 貴美子          1     女性  1962-03-12   57   
71   CS001515000096         水野 陽子          9     不明  1960-11-29   58   
122  CS013615000053         西脇 季衣          1     女性  1953-10-18   65   
144  CS020412000161          小宮 薫          1     女性  1974-05-21   44   
178  CS001215000097        竹中 あさみ          1     女性  1990-07-25   28   
252  CS035212000007        内村 恵梨香          1     女性  1990-12-04   28   
259  CS002515000386         野田 コウ          1     女性  1963-05-30   55   
293  CS001615000372        稲垣 寿々花          1     女性  1956-10-29   62   
297  CS032512000121         松井 知世          1     女性  1962-09-04   56   

    postal_cd                   address application_store_cd  \
12   223-0062  神奈川県横浜市港北区日吉本町**********               S14011   
68   249-0002      神奈川県逗子市山の根**********               S14022   
71   144-0053      東京都大田区蒲田本町**********               S13001   
122  261-0026    千葉県千葉市美浜区幕張西**********               S12013   
144  174-0042       東京都板橋区東坂下**********               S13020   
178  146-0095       東京都大田区多摩川**********               S13001   
252  152-0023        東京都目黒区八雲**********               S13035   
259  185-0013     東京都国分寺市西恋ケ窪**********               S13002   
293  144-0035       東京都大田区南蒲田**********               S13001   
297  210-0011   神奈川県川崎市川崎区富士見**********               S13032   

     application_date     status_cd  
12           20150228  C-20100421-9  
68           20150320  A-20091115-7  
71           20150614  A-20100724-7  
122          20150128  B-20100329-6  
144          20150822  B-20081021-3  
178          20170315  A-20100211-2  
252          20151013  B-20101018-6  
259          20160410  C-20100127-8  
293          20170403  A-20100104-1  
297          20150727  A-20100103-5

In [31]:
df_customer.query('status_cd.str.match("^[A-F].*[1-9]$")').head(10)# 同じ結果

customer_id customer_name  gender_cd gender   birth_day  age  \
12   CS011215000048         芦田 沙耶          1     女性  1992-02-01   27   
68   CS022513000105        島村 貴美子          1     女性  1962-03-12   57   
71   CS001515000096         水野 陽子          9     不明  1960-11-29   58   
122  CS013615000053         西脇 季衣          1     女性  1953-10-18   65   
144  CS020412000161          小宮 薫          1     女性  1974-05-21   44   
178  CS001215000097        竹中 あさみ          1     女性  1990-07-25   28   
252  CS035212000007        内村 恵梨香          1     女性  1990-12-04   28   
259  CS002515000386         野田 コウ          1     女性  1963-05-30   55   
293  CS001615000372        稲垣 寿々花          1     女性  1956-10-29   62   
297  CS032512000121         松井 知世          1     女性  1962-09-04   56   

    postal_cd                   address application_store_cd  \
12   223-0062  神奈川県横浜市港北区日吉本町**********               S14011   
68   249-0002      神奈川県逗子市山の根**********               S14022   
71   144-0053      東京都大田区蒲田本町**********               S13001   
122  261-0026    千葉県千葉市美浜区幕張西**********               S12013   
144  174-0042       東京都板橋区東坂下**********               S13020   
178  146-0095       東京都大田区多摩川**********               S13001   
252  152-0023        東京都目黒区八雲**********               S13035   
259  185-0013     東京都国分寺市西恋ケ窪**********               S13002   
293  144-0035       東京都大田区南蒲田**********               S13001   
297  210-0011   神奈川県川崎市川崎区富士見**********               S13032   

     application_date     status_cd  
12           20150228  C-20100421-9  
68           20150320  A-20091115-7  
71           20150614  A-20100724-7  
122          20150128  B-20100329-6  
144          20150822  B-20081021-3  
178          20170315  A-20100211-2  
252          20151013  B-20101018-6  
259          20160410  C-20100127-8  
293          20170403  A-20100104-1  
297          20150727  A-20100103-5

---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [241]:
df_store.query('tel_no.str.contains("[0-9]{3}-[0-9]{3}-[0-9]{4}")')
# {#}で直前の文字の繰り返し回数を指定

# 同等
df_store[df_store.tel_no.str.contains('[0-9]{3}-[0-9]{3}-[0-9]{4}')]

store_cd store_name  prefecture_cd prefecture              address  \
0    S12014       千草台店             12        千葉県      千葉県千葉市稲毛区千草台一丁目   
1    S13002       国分寺店             13        東京都         東京都国分寺市本多二丁目   
2    S14010        菊名店             14       神奈川県      神奈川県横浜市港北区菊名一丁目   
3    S14033       阿久和店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
4    S14036     相模原中央店             14       神奈川県        神奈川県相模原市中央二丁目   
7    S14040       長津田店             14       神奈川県  神奈川県横浜市緑区長津田みなみ台五丁目   
9    S14050      阿久和西店             14       神奈川県    神奈川県横浜市瀬谷区阿久和西一丁目   
11   S13052        森野店             13        東京都          東京都町田市森野三丁目   
12   S14028       二ツ橋店             14       神奈川県       神奈川県横浜市瀬谷区二ツ橋町   
16   S14012      本牧和田店             14       神奈川県        神奈川県横浜市中区本牧和田   
18   S14046       北山田店             14       神奈川県     神奈川県横浜市都筑区北山田一丁目   
19   S14022        逗子店             14       神奈川県         神奈川県逗子市逗子一丁目   
20   S14011      日吉本町店             14       神奈川県    神奈川県横浜市港北区日吉本町四丁目   
21   S13016       小金井店             13        東京都         東京都小金井市本町一丁目   
22   S14034      川崎野川店             14       神奈川県         神奈川県川崎市宮前区野川   
26   S14048      中川中央店             14       神奈川県    神奈川県横浜市都筑区中川中央二丁目   
27   S12007        佐倉店             12        千葉県            千葉県佐倉市上志津   
28   S14026     辻堂西海岸店             14       神奈川県      神奈川県藤沢市辻堂西海岸二丁目   
29   S13041       八王子店             13        東京都            東京都八王子市大塚   
31   S14049      川崎大師店             14       神奈川県      神奈川県川崎市川崎区中瀬三丁目   
32   S14023        川崎店             14       神奈川県      神奈川県川崎市川崎区本町二丁目   
33   S13018        清瀬店             13        東京都          東京都清瀬市松山一丁目   
35   S14027       南藤沢店             14       神奈川県           神奈川県藤沢市南藤沢   
36   S14021       伊勢原店             14       神奈川県       神奈川県伊勢原市伊勢原四丁目   
37   S14047       相模原店             14       神奈川県       神奈川県相模原市千代田六丁目   
38   S12013       習志野店             12        千葉県         千葉県習志野市芝園一丁目   
40   S14042       新山下店             14       神奈川県      神奈川県横浜市中区新山下二丁目   
42   S12030        八幡店             12        千葉県          千葉県市川市八幡三丁目   
44   S14025        大和店             14       神奈川県           神奈川県大和市下和田   
45   S14045        厚木店             14       神奈川県         神奈川県厚木市中町二丁目   
47   S12029        東野店             12        千葉県          千葉県浦安市東野一丁目   
49   S12053        高洲店             12        千葉県          千葉県浦安市高洲五丁目   
51   S14024        三田店             14       神奈川県      神奈川県川崎市多摩区三田四丁目   
52   S14006       葛が谷店             14       神奈川県        神奈川県横浜市都筑区葛が谷   

                      address_kana        tel_no  longitude  latitude  \
0           チバケンチバシイナゲクチグサダイイッチョウメ  043-123-4003   140.1180  35.63559   
1             トウキョウトコクブンジシホンダニチョウメ  042-123-4008   139.4802  35.70566   
2        カナガワケンヨコハマシコウホククキクナイッチョウメ  045-123-4032   139.6326  35.50049   
3        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4043   139.4961  35.45918   
4           カナガワケンサガミハラシチュウオウニチョウメ  042-123-4045   139.3716  35.57327   
7    カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ  045-123-4046   139.4994  35.52398   
9        カナガワケンヨコハマシセヤクアクワニシイッチョウメ  045-123-4053   139.4961  35.45918   
11             トウキョウトマチダシモリノサンチョウメ  042-123-4030   139.4383  35.55293   
12          カナガワケンヨコハマシセヤクフタツバシチョウ  045-123-4042   139.4963  35.46304   
16            カナガワケンヨコハマシナカクホンモクワダ  045-123-4034   139.6582  35.42156   
18      カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ  045-123-4049   139.5916  35.56189   
19               カナガワケンズシシズシイッチョウメ  046-123-4036   139.5789  35.29642   
20  カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ  045-123-4033   139.6316  35.54655   
21          トウキョウトコガネイシホンチョウイッチョウメ  042-123-4015   139.5094  35.70018   
22             カナガワケンカワサキシミヤマエクノガワ  044-123-4044   139.5998  35.57693   
26   カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ  045-123-4051   139.5758  35.54912   
27                    チバケンサクラシカミシヅ  043-123-4001   140.1452  35.71872   
28      カナガワケンフジサワシツジドウニシカイガンニチョウメ  046-123-4040   139.4466  35.32464   
29                トウキョウトハチオウジシオオツカ  042-123-4026   139.4235 

---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [36]:
df_customer.sort_values('birth_day',ascending=True).head(10)
# デフォルトでascending=Trueなので本当は不要

customer_id customer_name  gender_cd gender   birth_day  age  \
18817  CS003813000014        村山 菜々美          1     女性  1928-11-26   90   
12328  CS026813000004         吉村 朝陽          1     女性  1928-12-14   90   
15682  CS018811000003         熊沢 美里          1     女性  1929-01-07   90   
15302  CS027803000004         内村 拓郎          0     男性  1929-01-12   90   
1681   CS013801000003         天野 拓郎          0     男性  1929-01-15   90   
7511   CS001814000022         鶴田 里穂          1     女性  1929-01-28   90   
2378   CS016815000002         山元 美紀          1     女性  1929-02-22   90   
4680   CS009815000003         中田 里穂          1     女性  1929-04-08   89   
16070  CS005813000015        金谷 恵梨香          1     女性  1929-04-09   89   
6305   CS012813000013         宇野 南朋          1     女性  1929-04-09   89   

      postal_cd                 address application_store_cd  \
18817  182-0007     東京都調布市菊野台**********               S13003   
12328  251-0043   神奈川県藤沢市辻堂元町**********               S14026   
15682  204-0004      東京都清瀬市野塩**********               S13018   
15302  251-0031  神奈川県藤沢市鵠沼藤が谷**********               S14027   
1681   274-0824     千葉県船橋市前原東**********               S12013   
7511   144-0045     東京都大田区南六郷**********               S13001   
2378   184-0005     東京都小金井市桜町**********               S13016   
4680   154-0014     東京都世田谷区新町**********               S13009   
16070  165-0032      東京都中野区鷺宮**********               S13005   
6305   231-0806  神奈川県横浜市中区本牧町**********               S14012   

       application_date     status_cd  
18817          20160214  0-00000000-0  
12328          20150723  0-00000000-0  
15682          20150403  0-00000000-0  
15302          20151227  0-00000000-0  
1681           20160120  0-00000000-0  
7511           20161012  A-20090415-7  
2378           20150629  C-20090923-C  
4680           20150421  D-20091021-E  
16070          20150506  0-00000000-0  
6305           20150712  0-00000000-0

---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [38]:
df_customer.sort_values('birth_day',ascending=False).head(10)

customer_id customer_name  gender_cd gender   birth_day  age  \
15639  CS035114000004         大村 美里          1     女性  2007-11-25   11   
7468   CS022103000002        福山 はじめ          9     不明  2007-10-02   11   
10745  CS002113000009        柴田 真悠子          1     女性  2007-09-17   11   
19811  CS004115000014         松井 京子          1     女性  2007-08-09   11   
7039   CS002114000010          山内 遥          1     女性  2007-06-03   11   
3670   CS025115000002         小柳 夏希          1     女性  2007-04-18   11   
12493  CS002113000025        広末 まなみ          1     女性  2007-03-30   12   
15977  CS033112000003         長野 美紀          1     女性  2007-03-22   12   
5716   CS007115000006          福岡 瞬          1     女性  2007-03-10   12   
15097  CS014113000008         矢口 莉緒          1     女性  2007-03-05   12   

      postal_cd                  address application_store_cd  \
15639  156-0053       東京都世田谷区桜**********               S13035   
7468   249-0006      神奈川県逗子市逗子**********               S14022   
10745  184-0014    東京都小金井市貫井南町**********               S13002   
19811  165-0031      東京都中野区上鷺宮**********               S13004   
7039   184-0015    東京都小金井市貫井北町**********               S13002   
3670   245-0018  神奈川県横浜市泉区上飯田町**********               S14025   
12493  184-0015    東京都小金井市貫井北町**********               S13002   
15977  245-0051  神奈川県横浜市戸塚区名瀬町**********               S14033   
5716   285-0845      千葉県佐倉市西志津**********               S12007   
15097  260-0041   千葉県千葉市中央区東千葉**********               S12014   

       application_date     status_cd  
15639          20150619  6-20091205-6  
7468           20160909  0-00000000-0  
10745          20160304  0-00000000-0  
19811          20161120  1-20081231-1  
7039           20160920  6-20100510-1  
3670           20160116  D-20100913-D  
12493          20171030  0-00000000-0  
15977          20150606  0-00000000-0  
5716           20151118  F-20101016-F  
15097          20150622  3-20091108-6

---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [69]:
pd.concat([df_receipt[['customer_id','amount']],df_receipt['amount'].rank(method='min',ascending=False)\
           .rename('amount_rank')],axis=1).sort_values('amount_rank').head(10)
# 基本形：cancat([df1,df2])で結合。カラムを増やす→axis=1(rowを増やすaxis=0はデフォルトなので省略可）
# .rank()で順位付け。同値の場合の処理をmethodで規定。minだと最小の整数＝２位タイみたいな表示。
# この例ではrank用のカラムを作成して結合しているが、１行なのでpandas.Seriesとなる。
# Dataframeの場合はrenameメソッドで辞書型でrenameしたが、Seriesの場合はエラーとなる。１行しかないのでrename先を指定する必要がないため文法（データタイプ）が区別されている。
# rename('column_name')でOK。

customer_id  amount  amount_rank
1202   CS011415000006   10925          1.0
62317  ZZ000000000000    6800          2.0
54095  CS028605000002    5780          3.0
4632   CS015515000034    5480          4.0
72747  ZZ000000000000    5480          4.0
10320  ZZ000000000000    5480          4.0
97294  CS021515000089    5440          7.0
28304  ZZ000000000000    5440          7.0
92246  CS009415000038    5280          9.0
68553  CS040415000200    5280          9.0

In [70]:
# 模範解答は複数行
df_tmp = pd.concat([df_receipt[['customer_id', 'amount']] 
                     ,df_receipt['amount'].rank(method='min', ascending=False)], axis=1)
df_tmp.columns = ['customer_id', 'amount', 'ranking']
df_tmp.sort_values('ranking', ascending=True).head(10)
# こちらではrenameメソッドは使わずcolumns属性の再指定でカラム名を整えている。

customer_id  amount  ranking
1202   CS011415000006   10925      1.0
62317  ZZ000000000000    6800      2.0
54095  CS028605000002    5780      3.0
4632   CS015515000034    5480      4.0
72747  ZZ000000000000    5480      4.0
10320  ZZ000000000000    5480      4.0
97294  CS021515000089    5440      7.0
28304  ZZ000000000000    5440      7.0
92246  CS009415000038    5280      9.0
68553  CS040415000200    5280      9.0

---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [71]:
df_tmp = pd.concat([df_receipt[['customer_id', 'amount']] 
                     ,df_receipt['amount'].rank(method='first', ascending=False)], axis=1)
df_tmp.columns = ['customer_id', 'amount', 'ranking']
df_tmp.sort_values('ranking', ascending=True).head(10)
# rankメソッドのmethod引数のオプション（同値の処理）：
## min:　最小値
## max: 最大値
## average: 平均（デフォルト）
## first: 処理順　→ 同値でも別の整数が入る
## dense: minと同様だが、２位がふたつあっても次が３位となる

customer_id  amount  ranking
1202   CS011415000006   10925      1.0
62317  ZZ000000000000    6800      2.0
54095  CS028605000002    5780      3.0
4632   CS015515000034    5480      4.0
10320  ZZ000000000000    5480      5.0
72747  ZZ000000000000    5480      6.0
28304  ZZ000000000000    5440      7.0
97294  CS021515000089    5440      8.0
596    CS015515000083    5280      9.0
11275  CS017414000114    5280     10.0

---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [72]:
len(df_receipt)
# Dataframeのlen = データ(row)数

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [85]:
print(len(df_receipt.groupby('customer_id')),
      len(df_receipt['customer_id'].unique()))
# 同じ結果となる二つの書き方

8307 8307


---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [115]:
# groupby.xxx()で統計的処理とかが可能。xxxはmin,max,mean,sum,size,std,var 等
df_receipt[['store_cd','amount','quantity']].groupby('store_cd').sum().head(5)

amount  quantity
store_cd                  
S12007    638761      2099
S12013    787513      2425
S12014    725167      2358
S12029    794741      2555
S12030    684402      2403

In [98]:
df_receipt[['store_cd','amount','quantity']].groupby('store_cd',as_index=False).sum().head(5)
# デフォルトではグループラベルがindexになってテーブルが生成されるが、引数でas_index=Falseとするとindexが残る。

store_cd  amount  quantity
0   S12007  638761      2099
1   S12013  787513      2425
2   S12014  725167      2358
3   S12029  794741      2555
4   S12030  684402      2403

In [116]:
# しかし、groupby.agg()を使うとカッコ内の任意の処理が可能なためこれだけ覚えておいても対応可能。
df_receipt[['store_cd','amount','quantity']].groupby('store_cd').agg(sum).reset_index().head(5)
# reset_index()はas_index=Falseと同じ働き

store_cd  amount  quantity
0   S12007  638761      2099
1   S12013  787513      2425
2   S12014  725167      2358
3   S12029  794741      2555
4   S12030  684402      2403

In [118]:
# groupby.agg()の中身はリストにすると格計算を一気に出せる
df_receipt[['store_cd','amount','quantity']].groupby('store_cd').agg([sum,np.mean,max]).head(5)

amount                   quantity              
             sum        mean   max      sum      mean max
store_cd                                                 
S12007    638761  307.688343  5000     2099  1.011079   3
S12013    787513  330.194130  5000     2425  1.016771   3
S12014    725167  310.830261  5000     2358  1.010716   3
S12029    794741  315.623908  5280     2555  1.014694   3
S12030    684402  288.533727  5280     2403  1.013069   3

In [243]:
# groupby.agg()の中身は辞書型（カラム名をキー）とすることでカラム毎に違った処理をさせることができる
df_receipt[['store_cd','amount','quantity']].groupby('store_cd').agg({'amount':sum,'quantity':np.mean}).head(5)

amount  quantity
store_cd                  
S12007    638761  1.011079
S12013    787513  1.016771
S12014    725167  1.010716
S12029    794741  1.014694
S12030    684402  1.013069

In [109]:
# groupby.describeで分布に関する情報を一通り出すことができる。
df_receipt[['store_cd','amount','quantity']].groupby('store_cd').describe().head(5)

amount                                                             \
           count        mean         std   min    25%    50%    75%     max   
store_cd                                                                      
S12007    2076.0  307.688343  447.185532  10.0  102.0  160.0  278.0  5000.0   
S12013    2385.0  330.194130  470.268372  10.0  105.0  175.0  298.0  5000.0   
S12014    2333.0  310.830261  448.366601  10.0  102.0  165.0  288.0  5000.0   
S12029    2518.0  315.623908  440.631027  10.0  102.0  178.0  288.0  5280.0   
S12030    2372.0  288.533727  430.837734  10.0  100.0  165.0  280.0  5280.0   

         quantity                                               
            count      mean       std  min  25%  50%  75%  max  
store_cd                                                        
S12007     2076.0  1.011079  0.113531  1.0  1.0  1.0  1.0  3.0  
S12013     2385.0  1.016771  0.140900  1.0  1.0  1.0  1.0  3.0  
S12014     2333.0  1.010716  0.110999  1.0  1.0  1.0  1.0  3.0  
S12029     2518.0  1.014694  0.123607  1.0  1.0  1.0  1.0  3.0  
S12030     2372.0  1.013069  0.124235  1.0  1.0  1.0  1.0  3.0

In [121]:
# 模範解答
df_receipt.groupby('store_cd').agg({'amount':sum, 'quantity':sum}).reset_index().head(10)

store_cd  amount  quantity
0   S12007  638761      2099
1   S12013  787513      2425
2   S12014  725167      2358
3   S12029  794741      2555
4   S12030  684402      2403
5   S13001  811936      2347
6   S13002  727821      2340
7   S13003  764294      2197
8   S13004  779373      2390
9   S13005  629876      2004

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [128]:
df_receipt.groupby('customer_id', as_index=False).agg({'sales_ymd':max}).head(10)

customer_id  sales_ymd
0  CS001113000004   20190308
1  CS001114000005   20190731
2  CS001115000010   20190405
3  CS001205000004   20190625
4  CS001205000006   20190224
5  CS001211000025   20190322
6  CS001212000027   20170127
7  CS001212000031   20180906
8  CS001212000046   20170811
9  CS001212000070   20191018

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [129]:
df_receipt.groupby('customer_id', as_index=False).agg({'sales_ymd':min}).head(10)

customer_id  sales_ymd
0  CS001113000004   20190308
1  CS001114000005   20180503
2  CS001115000010   20171228
3  CS001205000004   20170914
4  CS001205000006   20180207
5  CS001211000025   20190322
6  CS001212000027   20170127
7  CS001212000031   20180906
8  CS001212000046   20170811
9  CS001212000070   20191018

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [169]:
# 引き算してゼロ以外を取得する方法。周りくどい
# agg内に関数を入れられるので定義してやる
def max_minus_min(x):
    y=max(x)-min(x)
    return(y)
    
df1=df_receipt[['customer_id','sales_ymd']].groupby('customer_id',as_index=False)\
.agg({'sales_ymd':[max,min,max_minus_min]})
df1[df1['sales_ymd','max_minus_min']!=0].head(10)

customer_id sales_ymd                        
                         max       min max_minus_min
1   CS001114000005  20190731  20180503         10228
2   CS001115000010  20190405  20171228         19177
3   CS001205000004  20190625  20170914         19711
4   CS001205000006  20190224  20180207         10017
13  CS001214000009  20190902  20170306         20596
14  CS001214000017  20191006  20180828         10178
16  CS001214000048  20190929  20171109         19820
17  CS001214000052  20190617  20180208         10409
20  CS001215000005  20181021  20170206         10815
21  CS001215000040  20171022  20170214           808

In [174]:
# これでいい
df1=df_receipt[['customer_id','sales_ymd']].groupby('customer_id',as_index=False)\
.agg({'sales_ymd':[max,min]})
df1[df1['sales_ymd','max']!=df1['sales_ymd','min']].head(10)

customer_id sales_ymd          
                         max       min
1   CS001114000005  20190731  20180503
2   CS001115000010  20190405  20171228
3   CS001205000004  20190625  20170914
4   CS001205000006  20190224  20180207
13  CS001214000009  20190902  20170306
14  CS001214000017  20191006  20180828
16  CS001214000048  20190929  20171109
17  CS001214000052  20190617  20180208
20  CS001215000005  20181021  20170206
21  CS001215000040  20171022  20170214

In [194]:
df1=df_receipt[['customer_id','sales_ymd']].groupby('customer_id',as_index=False)\
.agg({'sales_ymd':[max,min]})
df1['customer_id']
print('error')
# df1.query('customer_id != sales_ymd')#二行ヘッダーをうまく拾えない。やり方？

error


In [167]:
# 模範解答
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
df_tmp.columns = ["_".join(pair) for pair in df_tmp.columns]
df_tmp.query('sales_ymd_max != sales_ymd_min').head(10)
# ラベル行を一行にしている

customer_id_  sales_ymd_max  sales_ymd_min
1   CS001114000005       20190731       20180503
2   CS001115000010       20190405       20171228
3   CS001205000004       20190625       20170914
4   CS001205000006       20190224       20180207
13  CS001214000009       20190902       20170306
14  CS001214000017       20191006       20180828
16  CS001214000048       20190929       20171109
17  CS001214000052       20190617       20180208
20  CS001215000005       20181021       20170206
21  CS001215000040       20171022       20170214

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [208]:
df_receipt.groupby('store_cd').agg({'amount':np.mean}).reset_index()\
.sort_values('amount',ascending=False).head(5)

store_cd      amount
28   S13052  402.867470
12   S13015  351.111960
7    S13003  350.915519
30   S14010  348.791262
5    S13001  348.470386

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [209]:
df_receipt.groupby('store_cd').agg({'amount':np.median}).reset_index()\
.sort_values('amount',ascending=False).head(5)

store_cd  amount
28   S13052     190
30   S14010     188
51   S14050     185
44   S14040     180
7    S13003     180

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [250]:
# 模範解答
df_receipt.groupby('store_cd').product_cd.apply(lambda x: x.mode()).reset_index().head(5)

# s.mode(), df.mode() 

store_cd  level_1  product_cd
0   S12007        0  P060303001
1   S12013        0  P060303001
2   S12014        0  P060303001
3   S12029        0  P060303001
4   S12030        0  P060303001

In [251]:
# aggでも同じ？
df_receipt.groupby('store_cd').product_cd.agg(lambda x: x.mode()).reset_index().head(5)

store_cd  product_cd
0   S12007  P060303001
1   S12013  P060303001
2   S12014  P060303001
3   S12029  P060303001
4   S12030  P060303001

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [286]:
df_receipt.groupby('store_cd').agg({'amount':np.var}).reset_index().sort_values('amount',ascending=False).head(5)

store_cd         amount
28   S13052  441863.252526
31   S14011  306442.242432
42   S14034  297068.392740
5    S13001  295558.842618
12   S13015  295427.197086

In [284]:
# 模範解答
df_receipt.groupby('store_cd').amount.var(ddof=True).reset_index().sort_values('amount', ascending=False).head(5)
# Ref: http://sinhrks.hatenablog.com/entry/2014/11/27/232150

# pandasメソッドでの”var()"はデフォルトで不偏分散となっている。
# これを標本分散にするには(ddof=True)にする（デフォルトがFalse）
# 一方で、numpyメソッドのnp.var()はデフォルトが標準分散(ddof=True)であるという違いがある。
# ddofはDelta Degrees of Freedomの略らしい。

store_cd         amount
28   S13052  441863.252526
31   S14011  306442.242432
42   S14034  297068.392740
5    S13001  295558.842618
12   S13015  295427.197086

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [288]:
df_receipt.groupby('store_cd').amount.std(ddof=True).reset_index().sort_values('amount',ascending=False).head(5)

store_cd      amount
28   S13052  664.727954
31   S14011  553.572256
42   S14034  545.039808
5    S13001  543.653237
12   S13015  543.532149

In [289]:
df_receipt.groupby('store_cd').agg({'amount':np.std}).reset_index().sort_values('amount',ascending=False).head(5)

store_cd      amount
28   S13052  664.727954
31   S14011  553.572256
42   S14034  545.039808
5    S13001  543.653237
12   S13015  543.532149

---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [295]:
# コード例1
np.percentile(df_receipt['amount'], q=[25, 50, 75,100])

array([  102.,   170.,   288., 10925.])

In [296]:
# コード例2
df_receipt.amount.quantile(q=np.arange(5)/4)

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [302]:
df_a = df_receipt.groupby('store_cd').agg({'amount':np.mean}).reset_index()
df_a[df_a['amount']>=330]

store_cd      amount
1    S12013  330.194130
5    S13001  348.470386
7    S13003  350.915519
8    S13004  330.943949
12   S13015  351.111960
16   S13019  330.208616
17   S13020  337.879932
28   S13052  402.867470
30   S14010  348.791262
31   S14011  335.718333
38   S14026  332.340588
46   S14045  330.082073
48   S14047  330.077073

In [306]:
df_receipt.groupby('store_cd').agg({'amount':np.mean}).reset_index().query('amount>=330')

store_cd      amount
1    S12013  330.194130
5    S13001  348.470386
7    S13003  350.915519
8    S13004  330.943949
12   S13015  351.111960
16   S13019  330.208616
17   S13020  337.879932
28   S13052  402.867470
30   S14010  348.791262
31   S14011  335.718333
38   S14026  332.340588
46   S14045  330.082073
48   S14047  330.077073

In [ ]:
df_receipt.groupby('store_cd').agg({'amount':np.mean}).reset_index().query('amount>=330')

---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [312]:
# 自力解答：かなり微妙。Zから始まるものを除外〜の書き方を取り入れて後述
df_b = df_receipt.groupby('customer_id').agg({'amount':np.sum})\
.query('customer_id.str.contains("^[A-Y]")').reset_index()

np.mean(df_b['amount'])

2547.742234529256

In [315]:
# 模範解答：queryを使わない書き方
df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().mean()
# "~"を先頭に入れることで”除外”になるらしい。

2547.742234529256

In [316]:
# 模範解答：queryを使う書き方
df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().mean()
# query内にnot。自力回答にもそのまま採用可。

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [330]:
# 確認
a=df_receipt[~df_receipt.customer_id.str.startswith('Z')].groupby('customer_id').agg({'amount':np.sum})
print(type(a))
b=df_receipt[~df_receipt.customer_id.str.startswith('Z')].groupby('customer_id').amount.sum()
print(type(b))
# 上の出し方（agg）だとDFが返り値。下の出し方（.amount)だとSeriesが返り値。

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [333]:
df_c = df_receipt[~df_receipt.customer_id.str.startswith('Z')].groupby('customer_id')\
.agg({'amount':np.sum}).reset_index()

m = df_c.amount.mean()

df_c[df_c['amount']>=m].head(10)

customer_id  amount
2   CS001115000010    3044
4   CS001205000006    3337
13  CS001214000009    4685
14  CS001214000017    4132
17  CS001214000052    5639
21  CS001215000040    3496
30  CS001304000006    3726
32  CS001305000005    3485
33  CS001305000011    4370
53  CS001315000180    3300

In [ ]:
# 模範解答
amount_mean = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().mean()
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_amount_sum[df_amount_sum['amount'] >= amount_mean].head(10)

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [352]:
# pd.mergeで結合
pd.merge(df_receipt,df_store[['store_cd','store_name']],on='store_cd',how='inner').head(10)
# defaultでhow=inner

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1257206400   S14006         112               1   
1   20181116   1258329600   S14006         112               2   
2   20170118   1200614400   S14006        1162               1   
3   20190524   1274659200   S14006        1192               1   
4   20190419   1271635200   S14006         112               2   
5   20181119   1258588800   S14006        1152               2   
6   20171211   1228953600   S14006        1132               2   
7   20191021   1287619200   S14006        1112               2   
8   20170710   1215648000   S14006        1132               2   
9   20190805   1280966400   S14006         112               1   

      customer_id  product_cd  quantity  amount store_name  
0  CS006214000001  P070305012         1     158       葛が谷店  
1  ZZ000000000000  P080401001         1      48       葛が谷店  
2  CS006815000006  P050406035         1     220       葛が谷店  
3  CS006514000034  P060104003         1      80       葛が谷店  
4  ZZ000000000000  P060501002         1     148       葛が谷店  
5  ZZ000000000000  P050701001         1      88       葛が谷店  
6  CS006515000175  P090903001         1      80       葛が谷店  
7  CS006415000221  P040602001         1     405       葛が谷店  
8  CS006411000036  P090301051         1     330       葛が谷店  
9  CS006211000012  P050104001         1     115       葛が谷店

In [344]:
# pd.mergeの挙動確認
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K1', 'K3', 'K5', 'K7'],
                      'C': ['C1', 'C3', 'C5', 'C7'],
                      'D': ['D1', 'D3', 'D5', 'D7']},
                     index=[1, 3, 5, 7])

In [345]:
left

key   A   B
0  K0  A0  B0
1  K1  A1  B1
2  K2  A2  B2
3  K3  A3  B3

In [346]:
right

key   C   D
1  K1  C1  D1
3  K3  C3  D3
5  K5  C5  D5
7  K7  C7  D7

In [348]:
# on='カラム名'で結合。
pd.merge(left,right,on='key', how='inner')
# how='inner':内部結合　keyで共通しているrowのみ残る。

key   A   B   C   D
0  K1  A1  B1  C1  D1
1  K3  A3  B3  C3  D3

In [349]:
# how='outer':外部結合　keyで共通していないrowも残る。この場合、NaNが発生する。
pd.merge(left,right,on='key', how='outer')

key    A    B    C    D
0  K0   A0   B0  NaN  NaN
1  K1   A1   B1   C1   D1
2  K2   A2   B2  NaN  NaN
3  K3   A3   B3   C3   D3
4  K5  NaN  NaN   C5   D5
5  K7  NaN  NaN   C7   D7

In [350]:
# how='left':左結合　左側のdfのみkeyで共通していないrowも残る。
pd.merge(left,right,on='key', how='left')

key   A   B    C    D
0  K0  A0  B0  NaN  NaN
1  K1  A1  B1   C1   D1
2  K2  A2  B2  NaN  NaN
3  K3  A3  B3   C3   D3

In [351]:
# how='right':右結合　右側のdfのみkeyで共通していないrowも残る。
pd.merge(left,right,on='key', how='right')

key    A    B   C   D
0  K1   A1   B1  C1  D1
1  K3   A3   B3  C3  D3
2  K5  NaN  NaN  C5  D5
3  K7  NaN  NaN  C7  D7

In [362]:
# left_on, right_onでカラム名が共通でない場合も結合できる。
left_a=left.rename(columns={'key':'key_a'})
pd.merge(left_a,right,left_on='key_a',right_on='key',how='inner')
# ただしこの場合leftとright両方のカラムが残るため、.drop('key',axis=1)で片方だけ残すこともできる。

key_a   A   B key   C   D
0    K1  A1  B1  K1  C1  D1
1    K3  A3  B3  K3  C3  D3

In [341]:
pd.merge(df_receipt,df_store,on='store_cd').head(5)

sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no  \
0   20181103   1257206400   S14006         112               1   
1   20181116   1258329600   S14006         112               2   
2   20170118   1200614400   S14006        1162               1   
3   20190524   1274659200   S14006        1192               1   
4   20190419   1271635200   S14006         112               2   

      customer_id  product_cd  quantity  amount store_name  prefecture_cd  \
0  CS006214000001  P070305012         1     158       葛が谷店             14   
1  ZZ000000000000  P080401001         1      48       葛が谷店             14   
2  CS006815000006  P050406035         1     220       葛が谷店             14   
3  CS006514000034  P060104003         1      80       葛が谷店             14   
4  ZZ000000000000  P060501002         1     148       葛が谷店             14   

  prefecture        address         address_kana        tel_no  longitude  \
0       神奈川県  神奈川県横浜市都筑区葛が谷  カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633   
1       神奈川県  神奈川県横浜市都筑区葛が谷  カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633   
2       神奈川県  神奈川県横浜市都筑区葛が谷  カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633   
3       神奈川県  神奈川県横浜市都筑区葛が谷  カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633   
4       神奈川県  神奈川県横浜市都筑区葛が谷  カナガワケンヨコハマシツヅキククズガヤ  045-123-4031   139.5633   

   latitude  floor_area  
0  35.53573      1886.0  
1  35.53573      1886.0  
2  35.53573      1886.0  
3  35.53573      1886.0  
4  35.53573      1886.0

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [369]:
pd.merge(df_product,df_category[['category_major_cd','category_medium_cd',\
                                 'category_small_cd','category_small_name']],how='inner').head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost category_small_name  
0       198.0      149.0                 弁当類  
1       218.0      164.0                 弁当類  
2       230.0      173.0                 弁当類  
3       248.0      186.0                 弁当類  
4       268.0      201.0                 弁当類  
5       298.0      224.0                 弁当類  
6       338.0      254.0                 弁当類  
7       420.0      315.0                 弁当類  
8       498.0      374.0                 弁当類  
9       580.0      435.0                 弁当類

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [384]:
# 自力回答。模範回答と同じ結果ではある。outer結合した際にfillna()でカッコ内の値にNaNを置換できる。なぜかしなくても0になっている。
pd.merge(df_customer,df_receipt,how='outer').query('gender_cd==1 and not customer_id.str.startswith("Z")')\
.groupby('customer_id').agg({'amount':sum}).head(10)

amount
customer_id           
CS001112000009     0.0
CS001112000019     0.0
CS001112000021     0.0
CS001112000023     0.0
CS001112000024     0.0
CS001112000029     0.0
CS001112000030     0.0
CS001113000004  1298.0
CS001113000010     0.0
CS001114000005   626.0

In [386]:
# 模範回答の最後にsort_valuesを追加（自分の回答との結果を確認するため）。順番の違い
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_tmp = df_customer.query('gender_cd == "1" and not customer_id.str.startswith("Z")', engine='python')
pd.merge(df_tmp['customer_id'], df_amount_sum, how='left', on='customer_id').fillna(0).sort_values('customer_id').head(10)

customer_id  amount
11536  CS001112000009     0.0
7258   CS001112000019     0.0
6617   CS001112000021     0.0
8016   CS001112000023     0.0
16661  CS001112000024     0.0
3679   CS001112000029     0.0
10223  CS001112000030     0.0
3022   CS001113000004  1298.0
3633   CS001113000010     0.0
15586  CS001114000005   626.0

---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [436]:
# 自力回答。
df_count=df_receipt.drop_duplicates(subset=['customer_id', 'sales_ymd']).groupby('customer_id')\
.count().query('not customer_id.str.startswith("Z")').sort_values('sales_ymd',ascending=False)\
.reset_index().head(20)

df_sales=df_receipt.groupby('customer_id').agg({'amount':sum}).query('not customer_id.str.startswith("Z")')\
.sort_values('amount',ascending=False).reset_index().head(20)

pd.merge(df_count[['customer_id','sales_ymd']],df_sales[['customer_id','amount']],on='customer_id',how='outer')\
.sort_values(['amount','sales_ymd'],ascending=False).reset_index()

# drop_duplicated()メソッドで重複を削除できる。subset=[リスト]でリストの項目の完全一致を削除。

index     customer_id  sales_ymd   amount
0       5  CS017415000097       20.0  23086.0
1       1  CS015415000185       22.0  20153.0
2       7  CS031414000051       19.0  19202.0
3       4  CS028415000007       21.0  19127.0
4      20  CS001605000009        NaN  18925.0
5       2  CS010214000010       22.0  18585.0
6       6  CS016415000141       20.0  18372.0
7      21  CS006515000023        NaN  18372.0
8      22  CS011414000106        NaN  18338.0
9      23  CS038415000104        NaN  17847.0
10     24  CS035414000024        NaN  17615.0
11     25  CS021515000089        NaN  17580.0
12     26  CS032414000072        NaN  16563.0
13     27  CS016415000101        NaN  16348.0
14     28  CS011415000006        NaN  16094.0
15     29  CS034415000047        NaN  16083.0
16     30  CS007514000094        NaN  15735.0
17     31  CS009414000059        NaN  15492.0
18     32  CS030415000034        NaN  15468.0
19     33  CS015515000034        NaN  15300.0
20      0  CS040214000008       23.0      NaN
21      3  CS010214000002       21.0      NaN
22      8  CS014214000023       19.0      NaN
23      9  CS022515000226       19.0      NaN
24     10  CS021515000172       19.0      NaN
25     11  CS039414000052       19.0      NaN
26     12  CS021514000045       19.0      NaN
27     13  CS022515000028       18.0      NaN
28     14  CS030214000008       18.0      NaN
29     15  CS021515000056       18.0      NaN
30     16  CS014415000077       18.0      NaN
31     17  CS021515000211       18.0      NaN
32     18  CS032415000209       18.0      NaN
33     19  CS031414000073       18.0      NaN

In [438]:
# 模範回答
df_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_sum = df_sum.query('not customer_id.str.startswith("Z")', engine='python')
df_sum = df_sum.sort_values('amount', ascending=False).head(20)

df_cnt = df_receipt[~df_receipt.duplicated(subset=['customer_id', 'sales_ymd'])]
df_cnt = df_cnt.query('not customer_id.str.startswith("Z")', engine='python')
df_cnt = df_cnt.groupby('customer_id').sales_ymd.count().reset_index()
df_cnt = df_cnt.sort_values('sales_ymd', ascending=False).head(20)
# .duplicated()で重複のboolを取得する。デフォルトでは一つ目のみFalseで二つ目以降はTrueとなる。keep引数で設定できる。
pd.merge(df_sum, df_cnt, how='outer', on='customer_id')

customer_id   amount  sales_ymd
0   CS017415000097  23086.0       20.0
1   CS015415000185  20153.0       22.0
2   CS031414000051  19202.0       19.0
3   CS028415000007  19127.0       21.0
4   CS001605000009  18925.0        NaN
5   CS010214000010  18585.0       22.0
6   CS016415000141  18372.0       20.0
7   CS006515000023  18372.0        NaN
8   CS011414000106  18338.0        NaN
9   CS038415000104  17847.0        NaN
10  CS035414000024  17615.0        NaN
11  CS021515000089  17580.0        NaN
12  CS032414000072  16563.0        NaN
13  CS016415000101  16348.0        NaN
14  CS011415000006  16094.0        NaN
15  CS034415000047  16083.0        NaN
16  CS007514000094  15735.0        NaN
17  CS009414000059  15492.0        NaN
18  CS030415000034  15468.0        NaN
19  CS015515000034  15300.0        NaN
20  CS040214000008      NaN       23.0
21  CS010214000002      NaN       21.0
22  CS014214000023      NaN       19.0
23  CS022515000226      NaN       19.0
24  CS021515000172      NaN       19.0
25  CS039414000052      NaN       19.0
26  CS021514000045      NaN       19.0
27  CS022515000028      NaN       18.0
28  CS030214000008      NaN       18.0
29  CS021515000056      NaN       18.0
30  CS014415000077      NaN       18.0
31  CS021515000211      NaN       18.0
32  CS032415000209      NaN       18.0
33  CS031414000073      NaN       18.0

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [441]:
# 自力回答。答え一致してるし今回はこれで足りてる気がする。
len(df_store)*len(df_product)

531590

In [444]:
# 模範回答。謎の周りくどい方法。
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()
# 以下で'key'という名前のカラムを追加して全行の値にゼロを入れている。
df_store_tmp['key'] = 0
df_product_tmp['key'] = 0
len(pd.merge(df_store_tmp, df_product_tmp, how='outer', on='key'))

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [493]:
# 自力回答。模範回答と比べ0行目がNaNにできなかった。
df_output=df_receipt.groupby('sales_ymd').sum().reset_index()[['sales_ymd','amount']]
# 差分用のカラムを準備
df_output['dif']=0

# ループ処理(row単位）
# row毎にamountを取り出す。.iatメソッドでその行のamountと前日のamountを取り出して引いてdifに代入。
# i=0の時に.iat[-1,1]となり、最終行の値と計算してしまっていたので、ゼロのままにしておくようにした。実際には値無しが正しい。

for i,r in enumerate(df_output['sales_ymd']):
    if i==0:
        continue
    df_output.iat[i,2]=df_output.iat[i,1]-df_output.iat[i-1,1]

df_output.head(10)

sales_ymd  amount   dif
0   20170101   33723     0
1   20170102   24165 -9558
2   20170103   27503  3338
3   20170104   36165  8662
4   20170105   37830  1665
5   20170106   32387 -5443
6   20170107   23415 -8972
7   20170108   24737  1322
8   20170109   26718  1981
9   20170110   20143 -6575

In [494]:
# 模範回答
df_sales_amount_by_date = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
df_sales_amount_by_date = pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift()], axis=1)
df_sales_amount_by_date.columns = ['sales_ymd','amount','lag_ymd','lag_amount']
df_sales_amount_by_date['diff_amount'] = df_sales_amount_by_date['amount'] - df_sales_amount_by_date['lag_amount']
df_sales_amount_by_date.head(10)
# .shitf()ではデフォルトで一行下にずれる。この際最上行はNaNになり、最下行は削除される（このためサイズは変わらない）
# 引数：priods=nでずらし幅を指定。マイナスだと上にずらす。axis=1だと横にshiftする。

sales_ymd  amount     lag_ymd  lag_amount  diff_amount
0   20170101   33723         NaN         NaN          NaN
1   20170102   24165  20170101.0     33723.0      -9558.0
2   20170103   27503  20170102.0     24165.0       3338.0
3   20170104   36165  20170103.0     27503.0       8662.0
4   20170105   37830  20170104.0     36165.0       1665.0
5   20170106   32387  20170105.0     37830.0      -5443.0
6   20170107   23415  20170106.0     32387.0      -8972.0
7   20170108   24737  20170107.0     23415.0       1322.0
8   20170109   26718  20170108.0     24737.0       1981.0
9   20170110   20143  20170109.0     26718.0      -6575.0

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [498]:
df_42=df_receipt[['sales_ymd','amount']].groupby('sales_ymd').sum().reset_index()
df_42=pd.concat([df_42,df_42.shift(),df_42.shift(periods=2),df_42.shift(periods=3)],axis=1)
df_42.head(10)

sales_ymd  amount   sales_ymd   amount   sales_ymd   amount   sales_ymd  \
0   20170101   33723         NaN      NaN         NaN      NaN         NaN   
1   20170102   24165  20170101.0  33723.0         NaN      NaN         NaN   
2   20170103   27503  20170102.0  24165.0  20170101.0  33723.0         NaN   
3   20170104   36165  20170103.0  27503.0  20170102.0  24165.0  20170101.0   
4   20170105   37830  20170104.0  36165.0  20170103.0  27503.0  20170102.0   
5   20170106   32387  20170105.0  37830.0  20170104.0  36165.0  20170103.0   
6   20170107   23415  20170106.0  32387.0  20170105.0  37830.0  20170104.0   
7   20170108   24737  20170107.0  23415.0  20170106.0  32387.0  20170105.0   
8   20170109   26718  20170108.0  24737.0  20170107.0  23415.0  20170106.0   
9   20170110   20143  20170109.0  26718.0  20170108.0  24737.0  20170107.0   

    amount  
0      NaN  
1      NaN  
2      NaN  
3  33723.0  
4  24165.0  
5  27503.0  
6  36165.0  
7  37830.0  
8  32387.0  
9  23415.0

In [499]:
# 模範回答コード例1:縦持ちケース
df_sales_amount_by_date = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
for i in range(1, 4):
    if i == 1:
        df_lag = pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift(i)],axis=1)
    else:
        df_lag = df_lag.append(pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift(i)],axis=1))
df_lag.columns = ['sales_ymd', 'amount', 'lag_ymd', 'lag_amount']
df_lag.dropna().sort_values(['sales_ymd','lag_ymd']).head(10)

sales_ymd  amount     lag_ymd  lag_amount
1   20170102   24165  20170101.0     33723.0
2   20170103   27503  20170101.0     33723.0
2   20170103   27503  20170102.0     24165.0
3   20170104   36165  20170101.0     33723.0
3   20170104   36165  20170102.0     24165.0
3   20170104   36165  20170103.0     27503.0
4   20170105   37830  20170102.0     24165.0
4   20170105   37830  20170103.0     27503.0
4   20170105   37830  20170104.0     36165.0
5   20170106   32387  20170103.0     27503.0

In [500]:
# 模範回答コード例2:横持ちケース
df_sales_amount_by_date = df_receipt[['sales_ymd', 'amount']].groupby('sales_ymd').sum().reset_index()
for i in range(1, 4):
    if i == 1:
        df_lag = pd.concat([df_sales_amount_by_date, df_sales_amount_by_date.shift(i)],axis=1)
    else:
        df_lag = pd.concat([df_lag, df_sales_amount_by_date.shift(i)],axis=1)
df_lag.columns = ['sales_ymd', 'amount', 'lag_ymd_1', 'lag_amount_1', 'lag_ymd_2', 'lag_amount_2', 'lag_ymd_3', 'lag_amount_3']
df_lag.dropna().sort_values(['sales_ymd']).head(10)

sales_ymd  amount   lag_ymd_1  lag_amount_1   lag_ymd_2  lag_amount_2  \
3    20170104   36165  20170103.0       27503.0  20170102.0       24165.0   
4    20170105   37830  20170104.0       36165.0  20170103.0       27503.0   
5    20170106   32387  20170105.0       37830.0  20170104.0       36165.0   
6    20170107   23415  20170106.0       32387.0  20170105.0       37830.0   
7    20170108   24737  20170107.0       23415.0  20170106.0       32387.0   
8    20170109   26718  20170108.0       24737.0  20170107.0       23415.0   
9    20170110   20143  20170109.0       26718.0  20170108.0       24737.0   
10   20170111   24287  20170110.0       20143.0  20170109.0       26718.0   
11   20170112   23526  20170111.0       24287.0  20170110.0       20143.0   
12   20170113   28004  20170112.0       23526.0  20170111.0       24287.0   

     lag_ymd_3  lag_amount_3  
3   20170101.0       33723.0  
4   20170102.0       24165.0  
5   20170103.0       27503.0  
6   20170104.0       36165.0  
7   20170105.0       37830.0  
8   20170106.0       32387.0  
9   20170107.0       23415.0  
10  20170108.0       24737.0  
11  20170109.0       26718.0  
12  20170110.0       20143.0

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [525]:
# 自力回答。floorは小数点以下の切り捨てのため、桁を変えてfloorを掛けてから桁を戻した。
# 問題をちゃんと読んでなかったのでクロス集計になっていない。するためには下記の模範回答のようにpivot_tableを使う。
df_sum=pd.merge(df_receipt,df_customer,on='customer_id',how='inner')[['customer_id','amount','gender_cd','age']]
df_sum['age_range']=np.floor(df_sum['age']/10)*10
df_sum.groupby(['gender_cd','age_range']).agg({'amount':sum}).reset_index()

gender_cd  age_range   amount
0           0       10.0     1591
1           0       20.0    72940
2           0       30.0   177322
3           0       40.0    19355
4           0       50.0    54320
5           0       60.0   272469
6           0       70.0    13435
7           0       80.0    46360
8           1       10.0   149836
9           1       20.0  1363724
10          1       30.0   693047
11          1       40.0  9320791
12          1       50.0  6685192
13          1       60.0   987741
14          1       70.0    29764
15          1       80.0   262923
16          1       90.0     6260
17          9       10.0     4317
18          9       20.0    44328
19          9       30.0    50441
20          9       40.0   483512
21          9       50.0   342923
22          9       60.0    71418
23          9       70.0     2427
24          9       80.0     5111

In [559]:
# 模範回答。意外にも切り捨ての仕方は自力と同じ（便利な関数がありそうなものだけど）。
# pivot_table関数であるカラムの値をカラム名にして再構成できる。見た目が自力よりイケてる形に。
df_tmp = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df_tmp['era'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
df_sales_summary = pd.pivot_table(df_tmp, index='era', columns='gender_cd', values='amount', aggfunc=sum).reset_index()
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary
# pivot_tableの使い方　https://note.nkmk.me/python-pandas-pivot-table/
# indexで行ラベル、columnsで列ラベル、valuesで値を選択。デフォルトでは値の平均(np.mean)が出るが、aggfuncで指定できる。dropna=FalseにするとNaNも計算に入れる。
# aggfuncには関数名を入れられる。この場合はクォーテーション不要。更にカッコも不要。
# pivot_tableは並び替えと集計を行うが、stack(),unstack(),pivot()で単純に並び替えが可能

era      male     female   unknown
0   10    1591.0   149836.0    4317.0
1   20   72940.0  1363724.0   44328.0
2   30  177322.0   693047.0   50441.0
3   40   19355.0  9320791.0  483512.0
4   50   54320.0  6685192.0  342923.0
5   60  272469.0   987741.0   71418.0
6   70   13435.0    29764.0    2427.0
7   80   46360.0   262923.0    5111.0
8   90       NaN     6260.0       NaN

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [562]:
# 自力は難しかったので答え見ながら
df_sales_summary2=df_sales_summary.set_index('era').stack().reset_index().replace({'female':'01','male':'00','unknown':'99'})
df_sales_summary2.columns=['era','gender_cd','amount']
df_sales_summary2
# stack()するとpd.Seriesになる。reset_index()することでDataFrameに戻る。先にset_index()でeraを出しておく必要がある。

era gender_cd     amount
0    10        00     1591.0
1    10        01   149836.0
2    10        99     4317.0
3    20        00    72940.0
4    20        01  1363724.0
5    20        99    44328.0
6    30        00   177322.0
7    30        01   693047.0
8    30        99    50441.0
9    40        00    19355.0
10   40        01  9320791.0
11   40        99   483512.0
12   50        00    54320.0
13   50        01  6685192.0
14   50        99   342923.0
15   60        00   272469.0
16   60        01   987741.0
17   60        99    71418.0
18   70        00    13435.0
19   70        01    29764.0
20   70        99     2427.0
21   80        00    46360.0
22   80        01   262923.0
23   80        99     5111.0
24   90        01     6260.0

In [534]:
# 模範回答。
df_sales_summary = df_sales_summary.set_index('era'). \
        stack().reset_index().replace({'female':'01',
                                        'male':'00',
                                        'unknown':'99'}).rename(columns={'level_1':'gender_cd', 0: 'amount'})
df_sales_summary

era gender_cd     amount
0    10        00     1591.0
1    10        01   149836.0
2    10        99     4317.0
3    20        00    72940.0
4    20        01  1363724.0
5    20        99    44328.0
6    30        00   177322.0
7    30        01   693047.0
8    30        99    50441.0
9    40        00    19355.0
10   40        01  9320791.0
11   40        99   483512.0
12   50        00    54320.0
13   50        01  6685192.0
14   50        99   342923.0
15   60        00   272469.0
16   60        01   987741.0
17   60        99    71418.0
18   70        00    13435.0
19   70        01    29764.0
20   70        99     2427.0
21   80        00    46360.0
22   80        01   262923.0
23   80        99     5111.0
24   90        01     6260.0

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [583]:
# my回答
# pd.to_datetimeでフレーム内の文字列をdatetime型に変換可能。この際”文字列の”フォーマットを書いてやることでうまく変換できる。
# datetime型になったら任意の形に再変換してやれば好きなdatetime表現が可能。.dt属性で抜き出して.strftimeメソッドで変換できる。
s_datetime=pd.to_datetime(df_customer['birth_day'],format='%Y-%m-%d').dt.strftime('%Y%m%d')
df_customer2=df_customer.copy()
df_customer2['birth_day']=s_datetime
df_customer2[['customer_id','birth_day']].head(10)

customer_id birth_day
0  CS021313000114  19810429
1  CS037613000071  19520401
2  CS031415000172  19761004
3  CS028811000001  19330327
4  CS001215000145  19950329
5  CS020401000016  19740915
6  CS015414000103  19770809
7  CS029403000008  19730817
8  CS015804000004  19310502
9  CS033513000180  19620711

In [582]:
# 模範回答。代入ではなくconcatで実装している。
pd.concat([df_customer['customer_id'],
           pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')],
          axis = 1).head(10)

customer_id birth_day
0  CS021313000114  19810429
1  CS037613000071  19520401
2  CS031415000172  19761004
3  CS028811000001  19330327
4  CS001215000145  19950329
5  CS020401000016  19740915
6  CS015414000103  19770809
7  CS029403000008  19730817
8  CS015804000004  19310502
9  CS033513000180  19620711

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [589]:
# 逆変換。基本的に同じ。
df_46=pd.to_datetime(df_customer['application_date'],format='%Y%m%d')
df_46=pd.concat([df_customer['customer_id'],df_46],axis=1)
df_46.head(10)

customer_id application_date
0  CS021313000114       2015-09-05
1  CS037613000071       2015-04-14
2  CS031415000172       2015-05-29
3  CS028811000001       2016-01-15
4  CS001215000145       2017-06-05
5  CS020401000016       2015-02-25
6  CS015414000103       2015-07-22
7  CS029403000008       2015-05-15
8  CS015804000004       2015-06-07
9  CS033513000180       2015-07-28

In [590]:
# 模範回答。間違ってる（format指定してないからか変なことになってる）
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['application_date'])], axis=1).head(10)

customer_id              application_date
0  CS021313000114 1970-01-01 00:00:00.020150905
1  CS037613000071 1970-01-01 00:00:00.020150414
2  CS031415000172 1970-01-01 00:00:00.020150529
3  CS028811000001 1970-01-01 00:00:00.020160115
4  CS001215000145 1970-01-01 00:00:00.020170605
5  CS020401000016 1970-01-01 00:00:00.020150225
6  CS015414000103 1970-01-01 00:00:00.020150722
7  CS029403000008 1970-01-01 00:00:00.020150515
8  CS015804000004 1970-01-01 00:00:00.020150607
9  CS033513000180 1970-01-01 00:00:00.020150728

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [594]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_ymd']\
                                                                      ,format='%Y%m%d')],axis=1).head(10)

receipt_no  receipt_sub_no  sales_ymd
0         112               1 2018-11-03
1        1132               2 2018-11-18
2        1102               1 2017-07-12
3        1132               1 2019-02-05
4        1102               2 2018-08-21
5        1112               1 2019-06-05
6        1102               2 2018-12-05
7        1102               1 2019-09-22
8        1112               2 2017-05-04
9        1102               1 2019-10-10

In [606]:
# 模範回答。astypeしている。しないと前の問題の誤模範回答のようになる。format=でもうまくいっている。
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_ymd'].astype('str'))],axis=1).head(10)

receipt_no  receipt_sub_no  sales_ymd
0         112               1 2018-11-03
1        1132               2 2018-11-18
2        1102               1 2017-07-12
3        1132               1 2019-02-05
4        1102               2 2018-08-21
5        1112               1 2019-06-05
6        1102               2 2018-12-05
7        1102               1 2019-09-22
8        1112               2 2017-05-04
9        1102               1 2019-10-10

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [609]:
# to_datetimeの引数unit='s'でUNIX時間であることを定義する効果がある。
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch']\
                                                                      ,unit='s')],axis=1).head(10)

receipt_no  receipt_sub_no sales_epoch
0         112               1  2009-11-03
1        1132               2  2009-11-18
2        1102               1  2008-07-12
3        1132               1  2010-02-05
4        1102               2  2009-08-21
5        1112               1  2010-06-05
6        1102               2  2009-12-05
7        1102               1  2010-09-22
8        1112               2  2008-05-04
9        1102               1  2010-10-10

In [607]:
# 模範回答。
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s')],axis=1).head(10)

receipt_no  receipt_sub_no sales_epoch
0         112               1  2009-11-03
1        1132               2  2009-11-18
2        1102               1  2008-07-12
3        1132               1  2010-02-05
4        1102               2  2009-08-21
5        1112               1  2010-06-05
6        1102               2  2009-12-05
7        1102               1  2010-09-22
8        1112               2  2008-05-04
9        1102               1  2010-10-10

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [610]:
# dt属性から年や月の情報を取り出せる。
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch']\
                                                                      ,unit='s').dt.year],axis=1).head(10)

receipt_no  receipt_sub_no  sales_epoch
0         112               1         2009
1        1132               2         2009
2        1102               1         2008
3        1132               1         2010
4        1102               2         2009
5        1112               1         2010
6        1102               2         2009
7        1102               1         2010
8        1112               2         2008
9        1102               1         2010

In [ ]:
# 模範回答
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.year],axis=1).head(10)

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [613]:
# dt属性から年や月の情報を取り出せる。
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch']\
                                                                      ,unit='s').dt.strftime('%m')],axis=1).head(10)

receipt_no  receipt_sub_no sales_epoch
0         112               1          11
1        1132               2          11
2        1102               1          07
3        1132               1          02
4        1102               2          08
5        1112               1          06
6        1102               2          12
7        1102               1          09
8        1112               2          05
9        1102               1          10

In [614]:
# 模範回答。dt.monthでも月を取得できるが、ここでは0埋め２桁で取り出すためstrftimeを利用している
pd.concat([df_receipt[['receipt_no', 'receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_epoch'], unit='s').dt.strftime('%m')],axis=1).head(10)

receipt_no  receipt_sub_no sales_epoch
0         112               1          11
1        1132               2          11
2        1102               1          07
3        1132               1          02
4        1102               2          08
5        1112               1          06
6        1102               2          12
7        1102               1          09
8        1112               2          05
9        1102               1          10

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [615]:
# dt属性から年や月の情報を取り出せる。
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],pd.to_datetime(df_receipt['sales_epoch']\
                                                                      ,unit='s').dt.strftime('%d')],axis=1).head(10)

receipt_no  receipt_sub_no sales_epoch
0         112               1          03
1        1132               2          18
2        1102               1          12
3        1132               1          05
4        1102               2          21
5        1112               1          05
6        1102               2          05
7        1102               1          22
8        1112               2          04
9        1102               1          10

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [637]:
# 自力回答。結果は出せているが警告が出る。
df_52=df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id').agg({'amount':sum}).reset_index()
df_52['amount_binary']=df_52['amount']
df_52['amount_binary'][df_52['amount_binary']<=2000]=0
df_52['amount_binary'][df_52['amount_binary']>2000]=1
df_52.head(10)

/Users/yuya/.local/share/virtualenvs/test36--SgIQ7cC/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/yuya/.local/share/virtualenvs/test36--SgIQ7cC/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


customer_id  amount  amount_binary
0  CS001113000004    1298              0
1  CS001114000005     626              0
2  CS001115000010    3044              1
3  CS001205000004    1988              0
4  CS001205000006    3337              1
5  CS001211000025     456              0
6  CS001212000027     448              0
7  CS001212000031     296              0
8  CS001212000046     228              0
9  CS001212000070     456              0

In [625]:
# 模範回答。
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")', engine='python')
df_sales_amount = df_sales_amount[['customer_id', 'amount']].groupby('customer_id').sum().reset_index()
df_sales_amount['sales_flg'] = df_sales_amount['amount'].apply(lambda x: 1 if x > 2000 else 0)
df_sales_amount.head(10)

customer_id  amount  sales_flg
0  CS001113000004    1298          0
1  CS001114000005     626          0
2  CS001115000010    3044          1
3  CS001205000004    1988          0
4  CS001205000006    3337          1
5  CS001211000025     456          0
6  CS001212000027     448          0
7  CS001212000031     296          0
8  CS001212000046     228          0
9  CS001212000070     456          0

In [638]:
# lambda式が分かりにくいのでdefにした場合。
def def_2000cut(value):
    if value <=2000:
        value=0
        return(value)
    if value >2000:
        value=1
        return(value)
    
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")', engine='python')
df_sales_amount = df_sales_amount[['customer_id', 'amount']].groupby('customer_id').sum().reset_index()
df_sales_amount['sales_flg'] = df_sales_amount['amount'].apply(def_2000cut)
df_sales_amount.head(10)

customer_id  amount  sales_flg
0  CS001113000004    1298          0
1  CS001114000005     626          0
2  CS001115000010    3044          1
3  CS001205000004    1988          0
4  CS001205000006    3337          1
5  CS001211000025     456          0
6  CS001212000027     448          0
7  CS001212000031     296          0
8  CS001212000046     228          0
9  CS001212000070     456          0

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [651]:
# 自力回答。customer_idの重複はdrop_duplicates()で落としている。
df_53=df_customer.copy()
df_53['tokyo_1']=df_53['postal_cd'].apply(lambda x: 1 if 100<=int(x[0:3])<=209 else 0)
df_53_m=pd.merge(df_53[['customer_id','postal_cd','tokyo_1']],df_receipt[['customer_id']],on='customer_id',how='inner')
df_53_m.drop_duplicates('customer_id').groupby('tokyo_1').count()

customer_id  postal_cd
tokyo_1                        
0               3906       3906
1               4400       4400

In [652]:
# 模範回答。customer_idの重複はnuniqueでカウントすることでクリアしている。
df_tmp = df_customer[['customer_id', 'postal_cd']].copy()
df_tmp['postal_flg'] = df_tmp['postal_cd'].apply(lambda x: 1 if 100 <= int(x[0:3]) <= 209 else 0)

pd.merge(df_tmp, df_receipt, how='inner', on='customer_id'). \
    groupby('postal_flg').agg({'customer_id':'nunique'})

customer_id
postal_flg             
0                  3906
1                  4400

In [654]:
# ユニーク処理　https://note.nkmk.me/python-pandas-value-counts/

In [655]:
# Seriesからユニーク値のリスト抽出
df_53_m['tokyo_1'].unique()

array([1, 0])

In [656]:
# Seriesからユニーク値抽出と各値のカウント
df_53_m['tokyo_1'].value_counts()

0    34438
1    31244
Name: tokyo_1, dtype: int64

In [658]:
# SeriesやDataframeからユニークな値の数を抽出
#（値ごとのカウントがvalue_countでvalueの種類のカウントがnunique。len(s.unique())と一致するはず。
df_53_m.nunique()

customer_id    8306
postal_cd       923
tokyo_1           2
dtype: int64

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [667]:
# 自力回答。辞書を使った関数を定義。
def prefecture(address):
    pref_dict={'埼玉県':11,'千葉県':12,'東京都':13,'神奈川県':14}
    for key,value in pref_dict.items():# dict.items()でkeyとvalue両方を取得。つけないとkeyのみになる。
        if address.startswith(key):
            return value
# 関数テスト
prefecture('神奈川県伊勢原市')

14

In [668]:
df_54=df_customer[['customer_id','address']].copy()
df_54['prefecture_cd']=df_54['address'].apply(prefecture)
df_54.head(10)

customer_id                 address  prefecture_cd
0  CS021313000114    神奈川県伊勢原市粟窪**********             14
1  CS037613000071      東京都江東区南砂**********             13
2  CS031415000172     東京都渋谷区代々木**********             13
3  CS028811000001  神奈川県横浜市泉区和泉町**********             14
4  CS001215000145     東京都大田区仲六郷**********             13
5  CS020401000016      東京都板橋区若木**********             13
6  CS015414000103      東京都江東区北砂**********             13
7  CS029403000008      千葉県浦安市海楽**********             12
8  CS015804000004      東京都江東区北砂**********             13
9  CS033513000180  神奈川県横浜市旭区善部町**********             14

In [669]:
# 模範回答。map関数でより簡単に実装できている。
pd.concat([df_customer[['customer_id', 'address']], df_customer['address'].str[0:3].map({'埼玉県': '11',
                                                                           '千葉県':'12', 
                                                                           '東京都':'13', 
                                                                           '神奈川':'14'})], axis=1).head(10)

customer_id                 address address
0  CS021313000114    神奈川県伊勢原市粟窪**********      14
1  CS037613000071      東京都江東区南砂**********      13
2  CS031415000172     東京都渋谷区代々木**********      13
3  CS028811000001  神奈川県横浜市泉区和泉町**********      14
4  CS001215000145     東京都大田区仲六郷**********      13
5  CS020401000016      東京都板橋区若木**********      13
6  CS015414000103      東京都江東区北砂**********      13
7  CS029403000008      千葉県浦安市海楽**********      12
8  CS015804000004      東京都江東区北砂**********      13
9  CS033513000180  神奈川県横浜市旭区善部町**********      14

In [670]:
# map()メソッド　
# 使い方１：今回の例のように辞書型でkeyをvalueに置換する。.replace()と似ているが、mapだとkeyにない値はNaNにしてしまうので注意。
# https://note.nkmk.me/python-pandas-map-replace/
# 使い方２：applyと同様に引数に関数を入れて値を変換する。
# https://note.nkmk.me/python-pandas-map-applymap-apply/

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [3]:
df_55=df_receipt[['customer_id','amount']].copy()
df_55=df_55.groupby('customer_id').sum()
df_55['rank']=df_55['amount'].rank()

first=3*len(df_55['rank'])/4
secound=2*len(df_55['rank'])/4
third=len(df_55['rank'])/4
# カテゴリ１：rankがfirst以下、カテゴリ２：first > rank > second, カテゴリ３：secound > rank > third, カテゴリ４：third以上
df_55[df_55['rank']>=first].rank=1
df_55[df_55['rank']>=first].rank=1
# for rank in df_55['rank']:
#     if rank < len(df_55['rank'])/4:
#         rank = 4
#     elif rank < 2*len(df_55['rank'])/4:
#         rank=3
#     elif rank < 3*len(df_55['rank'])/4:
#         rank=2
#     else:
#         rank=1
print(len(df_55['rank']))
df_55

8307


amount    rank
customer_id                     
CS001113000004      1298  3828.5
CS001114000005       626  2327.5
CS001115000010      3044  5779.0
CS001205000004      1988  4738.0
CS001205000006      3337  6014.0
...                  ...     ...
CS051513000004       551  2085.5
CS051515000002       265   594.0
CS052212000002       192   173.0
CS052514000001       178   118.5
ZZ000000000000  12395003  8307.0

[8307 rows x 2 columns]

In [9]:
# 自力回答。defで"amount=1, return amount"としているが無駄なので"return 1"だけでよかった。
# 実装過程で最初a,b,c=...の部分もdefに入れたらめちゃ遅くなった。これは各行毎に計算しなければならなくなるからで、気を付けるべきこと。
df_55_2=df_receipt[['customer_id','amount']].copy()
df_55_2=df_55_2.groupby('customer_id').sum()
a,b,c = df_55_2['amount'].quantile([0.25,0.50,0.75])

def amount_categorizer(amount):
    if amount <= a:
        amount = 1
        return amount
    elif a<amount<=b:
        amount = 2
        return amount
    elif b<amount<=c:
        amount = 3
        return amount
    else:
        amount=4
        return amount

df_55_2['amount_category']=df_55_2['amount'].apply(amount_categorizer)
df_55_2.head(10)

amount  amount_category
customer_id                            
CS001113000004    1298                2
CS001114000005     626                2
CS001115000010    3044                3
CS001205000004    1988                3
CS001205000006    3337                3
CS001211000025     456                1
CS001212000027     448                1
CS001212000031     296                1
CS001212000046     228                1
CS001212000070     456                1

In [ ]:
# 模範回答：コード例1　np.quantileを使っている。自力側のs.quantileの方が短く書ける。
df_sales_amount = df_receipt[['customer_id', 'amount']].groupby('customer_id').sum().reset_index()
pct25 = np.quantile(df_sales_amount['amount'], 0.25)
pct50 = np.quantile(df_sales_amount['amount'], 0.5)
pct75 = np.quantile(df_sales_amount['amount'], 0.75)

def pct_group(x):
    if x < pct25:
        return 1
    elif pct25 <= x < pct50:
        return 2
    elif pct50 <= x < pct75:
        return 3
    elif pct75 <= x:
        return 4

df_sales_amount['pct_group'] = df_sales_amount['amount'].apply(lambda x: pct_group(x))
df_sales_amount.head(10)

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [16]:
# 自力回答。悪くないが、模範回答の書き方の方が短い
def age_class(age):
    age_tens=np.floor(age/10)*10
    if age_tens>60:
        age_tens=60
    return str(int(age_tens))+'s'

df_56=df_customer[['customer_id','birth_day','age']].copy()
df_56['generation']=df_56['age'].apply(age_class)
df_56.head(10)

customer_id   birth_day  age generation
0  CS021313000114  1981-04-29   37        30s
1  CS037613000071  1952-04-01   66        60s
2  CS031415000172  1976-10-04   42        40s
3  CS028811000001  1933-03-27   86        60s
4  CS001215000145  1995-03-29   24        20s
5  CS020401000016  1974-09-15   44        40s
6  CS015414000103  1977-08-09   41        40s
7  CS029403000008  1973-08-17   45        40s
8  CS015804000004  1931-05-02   87        60s
9  CS033513000180  1962-07-11   56        50s

In [18]:
# 模範回答。コード例1
# 自力回答と考え方は同じ。lambda式とminを使ってうまく書いている。
df_customer_era = pd.concat([df_customer[['customer_id', 'birth_day']],
                             df_customer['age'].apply(lambda x: min(math.floor(x / 10) * 10, 60))],
                             axis=1)

df_customer_era.head(10)

customer_id   birth_day  age
0  CS021313000114  1981-04-29   30
1  CS037613000071  1952-04-01   60
2  CS031415000172  1976-10-04   40
3  CS028811000001  1933-03-27   60
4  CS001215000145  1995-03-29   20
5  CS020401000016  1974-09-15   40
6  CS015414000103  1977-08-09   40
7  CS029403000008  1973-08-17   40
8  CS015804000004  1931-05-02   60
9  CS033513000180  1962-07-11   50

In [19]:
# 模範回答コード例2
# cutはbinningする関数である。
df_customer['age_group'] = pd.cut(df_customer['age'], bins=[0, 10, 20, 30, 40, 50, 60, np.inf], right=False)
df_customer[['customer_id', 'birth_day', 'age_group']].head(10)


customer_id   birth_day     age_group
0  CS021313000114  1981-04-29  [30.0, 40.0)
1  CS037613000071  1952-04-01   [60.0, inf)
2  CS031415000172  1976-10-04  [40.0, 50.0)
3  CS028811000001  1933-03-27   [60.0, inf)
4  CS001215000145  1995-03-29  [20.0, 30.0)
5  CS020401000016  1974-09-15  [40.0, 50.0)
6  CS015414000103  1977-08-09  [40.0, 50.0)
7  CS029403000008  1973-08-17  [40.0, 50.0)
8  CS015804000004  1931-05-02   [60.0, inf)
9  CS033513000180  1962-07-11  [50.0, 60.0)

In [17]:
# pd.cut()について　https://note.nkmk.me/python-pandas-cut-qcut-binning/

# pd.cut()の第一引数はリストやndarrayやSeries、第二引数はbinsの分け方。
# 第二引数の書き方１：分割数（整数）で最大値と最小値を等間隔に整数分割する。
# 書き方２：リスト指定で分割する（今回のコード例２の書き方）

---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [44]:
# 自力回答：gender_cdをstrの記号に変換してから結合した。intを数字のままstrに変換して使うケースは模範回答参照。
def def_57(gender_cd):
    if gender_cd==0:
        return 'M'
    if gender_cd==1:
        return 'F'
    else:
        return 'N'
    
df_57=pd.concat([df_56,df_customer['gender_cd'].apply(def_57)],axis=1)

df_57['cate56']=df_57['generation']+'_'+df_57['gender_cd']
df_57.head(10)

customer_id   birth_day  age generation gender_cd cate56
0  CS021313000114  1981-04-29   37        30s         F  30s_F
1  CS037613000071  1952-04-01   66        60s         N  60s_N
2  CS031415000172  1976-10-04   42        40s         F  40s_F
3  CS028811000001  1933-03-27   86        60s         F  60s_F
4  CS001215000145  1995-03-29   24        20s         F  20s_F
5  CS020401000016  1974-09-15   44        40s         M  40s_M
6  CS015414000103  1977-08-09   41        40s         F  40s_F
7  CS029403000008  1973-08-17   45        40s         M  40s_M
8  CS015804000004  1931-05-02   87        60s         M  60s_M
9  CS033513000180  1962-07-11   56        50s         F  50s_F

In [46]:
# 模範回答：簡単に済ませている。ポイントはseries同士を直接演算することができる点。
df_customer_era['era_gender'] = df_customer['gender_cd'].astype('str') + df_customer_era['age'].astype('str')
df_customer_era.head(2)

customer_id   birth_day  age era_gender
0  CS021313000114  1981-04-29   30        130
1  CS037613000071  1952-04-01   60        960

In [56]:
# 失敗例。型変換組み込み関数を使うと、pd.series自体が壊れて文字列になってしまう。
# .astype()メソッドであればseriesの状態で各要素が型変換される。
df_57a=df_56.copy()
df_57a['era_gender'] = str(df_57a['generation']) + str(df_customer['gender_cd'])
df_57a.head(2)

customer_id   birth_day  age generation  \
0  CS021313000114  1981-04-29   37        30s   
1  CS037613000071  1952-04-01   66        60s   

                                          era_gender  
0  0        30s\n1        60s\n2        40s\n3   ...  
1  0        30s\n1        60s\n2        40s\n3   ...

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [ ]:
# ダミー変数ってなんだよ？　→ https://note.nkmk.me/python-pandas-get-dummies/
# 要はカテゴリカルデータのカテゴリを全てカラム化してYes NO(One or Zero）にすること。
# ゼロとイチなので機械的処理がしやすい反面カラムが増えて可読性もなくなる。

In [57]:
pd.get_dummies(df_customer[['customer_id', 'gender_cd']], columns=['gender_cd']).head(10)

customer_id  gender_cd_0  gender_cd_1  gender_cd_9
0  CS021313000114            0            1            0
1  CS037613000071            0            0            1
2  CS031415000172            0            1            0
3  CS028811000001            0            1            0
4  CS001215000145            0            1            0
5  CS020401000016            1            0            0
6  CS015414000103            0            1            0
7  CS029403000008            1            0            0
8  CS015804000004            1            0            0
9  CS033513000180            0            1            0

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [64]:
df_59=df_receipt[['customer_id','amount']].query('not customer_id.str.startswith("Z")')\
.groupby('customer_id').sum().reset_index()
df_59['amount_std']=preprocessing.scale(df_59['amount'])
df_59.head(10)

customer_id  amount  amount_std
0  CS001113000004    1298   -0.459378
1  CS001114000005     626   -0.706390
2  CS001115000010    3044    0.182413
3  CS001205000004    1988   -0.205749
4  CS001205000006    3337    0.290114
5  CS001211000025     456   -0.768879
6  CS001212000027     448   -0.771819
7  CS001212000031     296   -0.827691
8  CS001212000046     228   -0.852686
9  CS001212000070     456   -0.768879

In [ ]:
# 標準化（平均==0, stdev==1)の方法　https://note.nkmk.me/python-list-ndarray-dataframe-normalize-standardize/

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [67]:
df_60=df_receipt[['customer_id','amount']].query('not customer_id.str.startswith("Z")')\
.groupby('customer_id').sum().reset_index()
df_60['amount_std']=preprocessing.minmax_scale(df_60['amount'])
df_60.head(10)

customer_id  amount  amount_std
0  CS001113000004    1298    0.053354
1  CS001114000005     626    0.024157
2  CS001115000010    3044    0.129214
3  CS001205000004    1988    0.083333
4  CS001205000006    3337    0.141945
5  CS001211000025     456    0.016771
6  CS001212000027     448    0.016423
7  CS001212000031     296    0.009819
8  CS001212000046     228    0.006865
9  CS001212000070     456    0.016771

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [71]:
# np.log10()とnp.power()。powerは指数計算
df_62=df_receipt[['customer_id','amount']].query('not customer_id.str.startswith("Z")')\
.groupby('customer_id').sum().reset_index()
df_62['amount_log10']=np.log10(df_62['amount'])
df_62['check']=np.power(10,df_62['amount_log10'])
df_62.head(10)

customer_id  amount  amount_log10   check
0  CS001113000004    1298      3.113275  1298.0
1  CS001114000005     626      2.796574   626.0
2  CS001115000010    3044      3.483445  3044.0
3  CS001205000004    1988      3.298416  1988.0
4  CS001205000006    3337      3.523356  3337.0
5  CS001211000025     456      2.658965   456.0
6  CS001212000027     448      2.651278   448.0
7  CS001212000031     296      2.471292   296.0
8  CS001212000046     228      2.357935   228.0
9  CS001212000070     456      2.658965   456.0

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [73]:
# np.log10()とnp.power()。powerは指数計算
df_62=df_receipt[['customer_id','amount']].query('not customer_id.str.startswith("Z")')\
.groupby('customer_id').sum().reset_index()
df_62['amount_loge']=np.log(df_62['amount'])
df_62['check']=np.power(np.e,df_62['amount_loge'])
df_62.head(10)

customer_id  amount  amount_loge   check
0  CS001113000004    1298     7.168580  1298.0
1  CS001114000005     626     6.439350   626.0
2  CS001115000010    3044     8.020928  3044.0
3  CS001205000004    1988     7.594884  1988.0
4  CS001205000006    3337     8.112827  3337.0
5  CS001211000025     456     6.122493   456.0
6  CS001212000027     448     6.104793   448.0
7  CS001212000031     296     5.690359   296.0
8  CS001212000046     228     5.429346   228.0
9  CS001212000070     456     6.122493   456.0

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [76]:
df_63=df_product.copy()
df_63['revenue']=df_63['unit_price']-df_63['unit_cost']
df_63.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  revenue  
0       198.0      149.0     49.0  
1       218.0      164.0     54.0  
2       230.0      173.0     57.0  
3       248.0      186.0     62.0  
4       268.0      201.0     67.0  
5       298.0      224.0     74.0  
6       338.0      254.0     84.0  
7       420.0      315.0    105.0  
8       498.0      374.0    124.0  
9       580.0      435.0    145.0

---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [85]:
# 自力回答。dropnaメソッドを使ってNaNを落としている
df_64=df_63.copy().dropna()
np.mean(df_64['revenue']/df_64['unit_price'])

0.24911389885177004

In [86]:
# 模範回答：meanメソッドのskipna=True
df_tmp = df_product.copy()
df_tmp['unit_profit_rate'] = (df_tmp['unit_price'] - df_tmp['unit_cost']) / df_tmp['unit_price']
df_tmp['unit_profit_rate'].mean(skipna=True)

0.24911389885177

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [88]:
# 自力回答
df_65=df_63.copy().dropna()
# unit_price - unit_cost = 0.3 * unit_price: unit_price = unit_cost/0.7
df_65['price_30']=df_65['unit_cost']/0.7
df_65['revenue_30']=(df_65['price_30']-df_65['unit_cost'])/df_65['price_30']
df_65.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  revenue    price_30  revenue_30  
0       198.0      149.0     49.0  212.857143         0.3  
1       218.0      164.0     54.0  234.285714         0.3  
2       230.0      173.0     57.0  247.142857         0.3  
3       248.0      186.0     62.0  265.714286         0.3  
4       268.0      201.0     67.0  287.142857         0.3  
5       298.0      224.0     74.0  320.000000         0.3  
6       338.0      254.0     84.0  362.857143         0.3  
7       420.0      315.0    105.0  450.000000         0.3  
8       498.0      374.0    124.0  534.285714         0.3  
9       580.0      435.0    145.0  621.428571         0.3

In [89]:
# 模範回答：math.floorはNaNでエラーとなるが、numpy.floorはエラーとならない
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x: np.floor(x / 0.7))
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  new_price  new_profit_rate  
0       198.0      149.0      212.0         0.297170  
1       218.0      164.0      234.0         0.299145  
2       230.0      173.0      247.0         0.299595  
3       248.0      186.0      265.0         0.298113  
4       268.0      201.0      287.0         0.299652  
5       298.0      224.0      320.0         0.300000  
6       338.0      254.0      362.0         0.298343  
7       420.0      315.0      450.0         0.300000  
8       498.0      374.0      534.0         0.299625  
9       580.0      435.0      621.0         0.299517

---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [93]:
# 自力回答: pdのround()
df_66=df_63.copy().dropna()
# unit_price - unit_cost = 0.3 * unit_price: unit_price = unit_cost/0.7
df_66['price_30']=(df_66['unit_cost']/0.7).round()
df_66['revenue_30']=(df_66['price_30']-df_66['unit_cost'])/df_66['price_30']
df_66.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  revenue  price_30  revenue_30  
0       198.0      149.0     49.0     213.0    0.300469  
1       218.0      164.0     54.0     234.0    0.299145  
2       230.0      173.0     57.0     247.0    0.299595  
3       248.0      186.0     62.0     266.0    0.300752  
4       268.0      201.0     67.0     287.0    0.299652  
5       298.0      224.0     74.0     320.0    0.300000  
6       338.0      254.0     84.0     363.0    0.300275  
7       420.0      315.0    105.0     450.0    0.300000  
8       498.0      374.0    124.0     534.0    0.299625  
9       580.0      435.0    145.0     621.0    0.299517

In [94]:
# 模範回答：np.round() 組み込みのroundはNaNでエラーとなるが、numpy.roundはエラーとならない
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x: np.round(x / 0.7))
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  new_price  new_profit_rate  
0       198.0      149.0      213.0         0.300469  
1       218.0      164.0      234.0         0.299145  
2       230.0      173.0      247.0         0.299595  
3       248.0      186.0      266.0         0.300752  
4       268.0      201.0      287.0         0.299652  
5       298.0      224.0      320.0         0.300000  
6       338.0      254.0      363.0         0.300275  
7       420.0      315.0      450.0         0.300000  
8       498.0      374.0      534.0         0.299625  
9       580.0      435.0      621.0         0.299517

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [96]:
# 自力回答: np.ceil()
df_66=df_63.copy().dropna()
# unit_price - unit_cost = 0.3 * unit_price: unit_price = unit_cost/0.7
df_66['price_30']=np.ceil(df_66['unit_cost']/0.7)
df_66['revenue_30']=(df_66['price_30']-df_66['unit_cost'])/df_66['price_30']
df_66.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  revenue  price_30  revenue_30  
0       198.0      149.0     49.0     213.0    0.300469  
1       218.0      164.0     54.0     235.0    0.302128  
2       230.0      173.0     57.0     248.0    0.302419  
3       248.0      186.0     62.0     266.0    0.300752  
4       268.0      201.0     67.0     288.0    0.302083  
5       298.0      224.0     74.0     320.0    0.300000  
6       338.0      254.0     84.0     363.0    0.300275  
7       420.0      315.0    105.0     451.0    0.301552  
8       498.0      374.0    124.0     535.0    0.300935  
9       580.0      435.0    145.0     622.0    0.300643

In [97]:
# 模範回答：math.ceilはNaNでエラーとなるが、numpy.ceilはエラーとならない
df_tmp = df_product.copy()
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x: np.ceil(x / 0.7))
df_tmp['new_profit_rate'] = (df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  new_price  new_profit_rate  
0       198.0      149.0      213.0         0.300469  
1       218.0      164.0      235.0         0.302128  
2       230.0      173.0      248.0         0.302419  
3       248.0      186.0      266.0         0.300752  
4       268.0      201.0      288.0         0.302083  
5       298.0      224.0      320.0         0.300000  
6       338.0      254.0      363.0         0.300275  
7       420.0      315.0      451.0         0.301552  
8       498.0      374.0      535.0         0.300935  
9       580.0      435.0      622.0         0.300643

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [100]:
df_68=df_product.copy().dropna()
df_68['price_with_tax']=np.floor(df_68['unit_price']*1.1)
df_68.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  price_with_tax  
0       198.0      149.0           217.0  
1       218.0      164.0           239.0  
2       230.0      173.0           253.0  
3       248.0      186.0           272.0  
4       268.0      201.0           294.0  
5       298.0      224.0           327.0  
6       338.0      254.0           371.0  
7       420.0      315.0           462.0  
8       498.0      374.0           547.0  
9       580.0      435.0           638.0

In [101]:
# math.floorはNaNでエラーとなるが、numpy.floorはエラーとならない
df_tmp = df_product.copy()
df_tmp['price_tax'] = df_tmp['unit_price'].apply(lambda x: np.floor(x * 1.1))
df_tmp.head(10)

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   
5  P040101006                  4                 401              40101   
6  P040101007                  4                 401              40101   
7  P040101008                  4                 401              40101   
8  P040101009                  4                 401              40101   
9  P040101010                  4                 401              40101   

   unit_price  unit_cost  price_tax  
0       198.0      149.0      217.0  
1       218.0      164.0      239.0  
2       230.0      173.0      253.0  
3       248.0      186.0      272.0  
4       268.0      201.0      294.0  
5       298.0      224.0      327.0  
6       338.0      254.0      371.0  
7       420.0      315.0      462.0  
8       498.0      374.0      547.0  
9       580.0      435.0      638.0

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [119]:
# 自力回答：
df_69=pd.merge(df_receipt,df_product,on='product_cd',how='inner').groupby('customer_id')\
.agg({'amount':sum}).reset_index()
df_69a=pd.merge(df_receipt,df_product,on='product_cd',how='inner').query('category_major_cd==7')\
.groupby('customer_id').agg({'amount':sum}).rename(columns={'amount':'07_amount'}).reset_index()
df_69b=pd.merge(df_69,df_69a,on='customer_id',how='inner')
df_69b['07_ratio']=df_69b['07_amount']/df_69b['amount']
df_69b.head(10)

customer_id  amount  07_amount  07_ratio
0  CS001113000004    1298       1298  1.000000
1  CS001114000005     626        486  0.776358
2  CS001115000010    3044       2694  0.885020
3  CS001205000004    1988        346  0.174044
4  CS001205000006    3337       2004  0.600539
5  CS001212000027     448        200  0.446429
6  CS001212000031     296        296  1.000000
7  CS001212000046     228        108  0.473684
8  CS001212000070     456        308  0.675439
9  CS001213000018     243        145  0.596708

In [120]:
# 模範回答：コード例1　自力と全く同じ
df_tmp_1 = pd.merge(df_receipt, df_product, 
                    how='inner', on='product_cd').groupby('customer_id').agg({'amount':'sum'}).reset_index()

df_tmp_2 = pd.merge(df_receipt, df_product.query('category_major_cd == "07"'), 
                    how='inner', on='product_cd').groupby('customer_id').agg({'amount':'sum'}).reset_index()

df_tmp_3 = pd.merge(df_tmp_1, df_tmp_2, how='inner', on='customer_id')
df_tmp_3['rate_07'] = df_tmp_3['amount_y'] / df_tmp_3['amount_x']
df_tmp_3.head(10)

customer_id  amount_x  amount_y   rate_07
0  CS001113000004      1298      1298  1.000000
1  CS001114000005       626       486  0.776358
2  CS001115000010      3044      2694  0.885020
3  CS001205000004      1988       346  0.174044
4  CS001205000006      3337      2004  0.600539
5  CS001212000027       448       200  0.446429
6  CS001212000031       296       296  1.000000
7  CS001212000046       228       108  0.473684
8  CS001212000070       456       308  0.675439
9  CS001213000018       243       145  0.596708

In [121]:
# 模範回答：コード例2
df_temp = df_receipt.merge(df_product, how='left', on='product_cd').groupby(['customer_id', 'category_major_cd'])['amount'].sum().unstack()
df_temp = df_temp[df_temp[7] > 0]
df_temp['sum'] = df_temp.sum(axis=1)
df_temp['07_rate'] = df_temp[7] / df_temp['sum']
df_temp.head(10)

category_major_cd      4      5      6       7      8      9     sum   07_rate
customer_id                                                                   
CS001113000004       NaN    NaN    NaN  1298.0    NaN    NaN  1298.0  1.000000
CS001114000005       NaN   40.0    NaN   486.0  100.0    NaN   626.0  0.776358
CS001115000010       NaN    NaN    NaN  2694.0    NaN  350.0  3044.0  0.885020
CS001205000004     100.0  128.0  286.0   346.0  368.0  760.0  1988.0  0.174044
CS001205000006     635.0   60.0  198.0  2004.0   80.0  360.0  3337.0  0.600539
CS001212000027     248.0    NaN    NaN   200.0    NaN    NaN   448.0  0.446429
CS001212000031       NaN    NaN    NaN   296.0    NaN    NaN   296.0  1.000000
CS001212000046       NaN    NaN    NaN   108.0    NaN  120.0   228.0  0.473684
CS001212000070       NaN    NaN  148.0   308.0    NaN    NaN   456.0  0.675439
CS001213000018       NaN    NaN    NaN   145.0   98.0    NaN   243.0  0.596708

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [134]:
# 自力回答：datetime同士で引き算したところ勝手にdaysになった
df_70=pd.merge(df_customer,df_receipt,on='customer_id',how='inner')
s_sales_ymd=pd.to_datetime(df_70['sales_ymd'],format='%Y%m%d')
s_app_date=pd.to_datetime(df_70['application_date'],format='%Y%m%d')
s_days=s_sales_ymd-s_app_date
df_70['days_from_application']=s_days
df_70[['customer_id','sales_ymd','application_date','days_from_application']].head(10)

customer_id  sales_ymd  application_date days_from_application
0  CS031415000172   20170507          20150529              709 days
1  CS031415000172   20171026          20150529              881 days
2  CS031415000172   20190325          20150529             1396 days
3  CS031415000172   20170111          20150529              593 days
4  CS031415000172   20190325          20150529             1396 days
5  CS031415000172   20190122          20150529             1334 days
6  CS031415000172   20170111          20150529              593 days
7  CS031415000172   20171026          20150529              881 days
8  CS031415000172   20190122          20150529             1334 days
9  CS031415000172   20171009          20150529              864 days

In [135]:
# 模範回答：mergeする前に行を減らして効率化。drop_duplicates()して同じ日の同じ人の買い物も減らしている。
# ただし肝心のdatetime変換がうまくいっていない
df_tmp = pd.merge(df_receipt[['customer_id', 'sales_ymd']], df_customer[['customer_id', 'application_date']],
                 how='inner', on='customer_id')

df_tmp = df_tmp.drop_duplicates()

df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
df_tmp['elapsed_date'] = df_tmp['sales_ymd'] - df_tmp['application_date']
df_tmp.head(10)

customer_id  sales_ymd              application_date  \
0   CS006214000001 2018-11-03 1970-01-01 00:00:00.020150201   
1   CS006214000001 2017-05-09 1970-01-01 00:00:00.020150201   
2   CS006214000001 2017-06-08 1970-01-01 00:00:00.020150201   
4   CS006214000001 2018-10-28 1970-01-01 00:00:00.020150201   
7   CS006214000001 2019-09-08 1970-01-01 00:00:00.020150201   
8   CS006214000001 2018-01-31 1970-01-01 00:00:00.020150201   
9   CS006214000001 2017-07-05 1970-01-01 00:00:00.020150201   
10  CS006214000001 2018-11-10 1970-01-01 00:00:00.020150201   
12  CS006214000001 2019-04-10 1970-01-01 00:00:00.020150201   
15  CS006214000001 2019-06-01 1970-01-01 00:00:00.020150201   

                    elapsed_date  
0  17837 days 23:59:59.979849799  
1  17294 days 23:59:59.979849799  
2  17324 days 23:59:59.979849799  
4  17831 days 23:59:59.979849799  
7  18146 days 23:59:59.979849799  
8  17561 days 23:59:59.979849799  
9  17351 days 23:59:59.979849799  
10 17844 days 23:59:59.979849799  
12 17995 days 23:59:59.979849799  
15 18047 days 23:59:59.979849799

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [177]:
# 重要：relativedeltaでないとmonthとかの計算はできない。datetimeの限界
# 重要：.apply(lambda x:)で各列の要素はリストで取り出せる！！(下で検証）


df_71=pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],\
               on='customer_id',how='inner').drop_duplicates()
# ↓だとdaysが出てしまう。30日で割ったところで正確なmonthにはならない。
#df_71['months_from_app']=pd.to_datetime(df_71['sales_ymd'],format='%Y%m%d')\
#-pd.to_datetime(df_71['application_date'],format='%Y%m%d')

df_71['sales_ymd']=pd.to_datetime(df_71['sales_ymd'],format='%Y%m%d')
df_71['application_date']=pd.to_datetime(df_71['application_date'],format='%Y%m%d')

df_71['months_from_app']=df_71[['sales_ymd','application_date']]\
.apply(lambda x: relativedelta(x[0],x[1]).years*12 + relativedelta(x[0],x[1]).months,axis=1)
df_71.head(10)

customer_id  sales_ymd application_date  months_from_app
0   CS006214000001 2018-11-03       2015-02-01               45
1   CS006214000001 2017-05-09       2015-02-01               27
2   CS006214000001 2017-06-08       2015-02-01               28
4   CS006214000001 2018-10-28       2015-02-01               44
7   CS006214000001 2019-09-08       2015-02-01               55
8   CS006214000001 2018-01-31       2015-02-01               35
9   CS006214000001 2017-07-05       2015-02-01               29
10  CS006214000001 2018-11-10       2015-02-01               45
12  CS006214000001 2019-04-10       2015-02-01               50
15  CS006214000001 2019-06-01       2015-02-01               52

### .apply(lambda x: hoge)の検証

In [225]:
# 検証：.apply(lambda x: hoge, axis=1)からx[数字]で各カラム要素を取り出せる
df_71a=df_71.head()#処理軽くするため
df_71b=df_71a.copy()
df_71b

customer_id  sales_ymd application_date  months_from_app
0  CS006214000001 2018-11-03       2015-02-01               45
1  CS006214000001 2017-05-09       2015-02-01               27
2  CS006214000001 2017-06-08       2015-02-01               28
4  CS006214000001 2018-10-28       2015-02-01               44
7  CS006214000001 2019-09-08       2015-02-01               55

In [221]:
# 検証: .apply(lambda x: hoge ,axis=1)のxはrowのSeriesである。
# つまり、xは各行の['customer_id','sales_ymd','application_date','months_from_app']の列ラベルと値を持っている。
df_71c=df_71a.copy()
df_71c['test2']=df_71c.apply(lambda x: str(type(x)),axis=1)
df_71c

customer_id  sales_ymd application_date  months_from_app  \
0  CS006214000001 2018-11-03       2015-02-01               45   
1  CS006214000001 2017-05-09       2015-02-01               27   
2  CS006214000001 2017-06-08       2015-02-01               28   
4  CS006214000001 2018-10-28       2015-02-01               44   
7  CS006214000001 2019-09-08       2015-02-01               55   

                                 test2  
0  <class 'pandas.core.series.Series'>  
1  <class 'pandas.core.series.Series'>  
2  <class 'pandas.core.series.Series'>  
4  <class 'pandas.core.series.Series'>  
7  <class 'pandas.core.series.Series'>

In [222]:
# default(axis=0)ではxはカラムのSeriesである。（あまり使わない？）
# つまり、xは各列のindexと値を持っている['0','1','2','4','7']
df_71d=df_71a.copy()
df_71d.loc['100']=df_71c.apply(lambda x: str(type(x)))
df_71d

customer_id                            sales_ymd  \
0                         CS006214000001                  2018-11-03 00:00:00   
1                         CS006214000001                  2017-05-09 00:00:00   
2                         CS006214000001                  2017-06-08 00:00:00   
4                         CS006214000001                  2018-10-28 00:00:00   
7                         CS006214000001                  2019-09-08 00:00:00   
100  <class 'pandas.core.series.Series'>  <class 'pandas.core.series.Series'>   

                        application_date                      months_from_app  
0                    2015-02-01 00:00:00                                   45  
1                    2015-02-01 00:00:00                                   27  
2                    2015-02-01 00:00:00                                   28  
4                    2015-02-01 00:00:00                                   44  
7                    2015-02-01 00:00:00                                   55  
100  <class 'pandas.core.series.Series'>  <class 'pandas.core.series.Series'>

In [226]:
# 検証：.apply(lambda x: hoge, axis=1)からx[列番号]で各カラム要素を取り出せる。
# これを使うと各行の値を使った計算が大体できる。
df_71e=df_71a.copy()
df_71e['test1']=df_71e.apply(lambda x: f'{str(x[0])}_{str(x[1])}_{str(x[3])}',axis=1)
df_71e

customer_id  sales_ymd application_date  months_from_app  \
0  CS006214000001 2018-11-03       2015-02-01               45   
1  CS006214000001 2017-05-09       2015-02-01               27   
2  CS006214000001 2017-06-08       2015-02-01               28   
4  CS006214000001 2018-10-28       2015-02-01               44   
7  CS006214000001 2019-09-08       2015-02-01               55   

                                   test1  
0  CS006214000001_2018-11-03 00:00:00_45  
1  CS006214000001_2017-05-09 00:00:00_27  
2  CS006214000001_2017-06-08 00:00:00_28  
4  CS006214000001_2018-10-28 00:00:00_44  
7  CS006214000001_2019-09-08 00:00:00_55

In [230]:
# 検証：カラム名でも取り出せる！！
df_71d=df_71a.copy()
df_71d.apply(lambda x: x["months_from_app"]+100,axis=1)

0    145
1    127
2    128
4    144
7    155
dtype: int64

In [232]:
# 検証：lambdaではなくdefでも同様
df_71d=df_71a.copy()
def test4(x):
    return x['months_from_app']+100
df_71d.apply(test4,axis=1)

0    145
1    127
2    128
4    144
7    155
dtype: int64

In [233]:
# x(axis=0)がindex:列要素のSeriesであるという動きの確認
df_71d.apply(lambda x: x[0:2])

customer_id  sales_ymd application_date  months_from_app
0  CS006214000001 2018-11-03       2015-02-01               45
1  CS006214000001 2017-05-09       2015-02-01               27

In [234]:
# x(axis=1)がcolumns:行要素のSeriesであるという動きの確認
df_71d.apply(lambda x: x[0:2],axis=1)

customer_id  sales_ymd
0  CS006214000001 2018-11-03
1  CS006214000001 2017-05-09
2  CS006214000001 2017-06-08
4  CS006214000001 2018-10-28
7  CS006214000001 2019-09-08

## dateutil.relativedelta.relativedelta の検証
#### dateutilはdatetimeでできないことを実装している外部ライブラリ

In [242]:
# テスト用
from datetime import datetime, date, timedelta
today=datetime.today()

In [243]:
today.strftime('%Y-%m-%d')

'2020-09-13'

In [245]:
today+timedelta(days=1)
# datetime.timedeltaで可能な引数は以下。monthsやyearsがないため、dateutilを使う
#...     days=50,
#...     seconds=27,
#...     microseconds=10,
#...     milliseconds=29000,
#...     minutes=5,
#...     hours=8,
#...     weeks=2

datetime.datetime(2020, 9, 14, 15, 59, 41, 794455)

In [248]:
from dateutil.relativedelta import relativedelta
today + relativedelta(months=1)
# relativedeltaはtimedeltaに加えてmonthsやyearsも実装されているので、timedeltaを兼ねた利用ができる。

datetime.datetime(2020, 10, 13, 15, 59, 41, 794455)

In [250]:
# 便利な使い方：今月の末日を出す方法
last_day = (today + relativedelta(months=1)).replace(day=1) - relativedelta(days=1)
print(datetime.strftime(last_day, '%Y-%m-%d'))

2020-09-30


In [261]:
someday=today+relativedelta(years=5,months=4,days=6)
# 二つのdatetimeの差分
delta=relativedelta(someday,today)
delta

relativedelta(years=+5, months=+4, days=+6)

In [264]:
delta.years

5

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [269]:
df_72=pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']]\
               ,on='customer_id',how='inner').drop_duplicates().reset_index(drop=True)#ないとまえのindexも残る

# pipe()を使ってみた。とても便利。引数はカッコなしで関数名を入れて、df以外の引数を第二引数以降に入れる。
df_72['sales_ymd']=df_72['sales_ymd'].pipe(pd.to_datetime,format='%Y%m%d')
df_72['application_date']=df_72['application_date'].pipe(pd.to_datetime,format='%Y%m%d')

df_72['years']=df_72[['sales_ymd','application_date']].apply(lambda x: relativedelta(x[0],x[1]).years,axis=1)
df_72.head()

customer_id  sales_ymd application_date  years
0  CS006214000001 2018-11-03       2015-02-01      3
1  CS006214000001 2017-05-09       2015-02-01      2
2  CS006214000001 2017-06-08       2015-02-01      2
3  CS006214000001 2018-10-28       2015-02-01      3
4  CS006214000001 2019-09-08       2015-02-01      4

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [274]:
df_73=pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']]\
               ,on='customer_id',how='inner').drop_duplicates().reset_index(drop=True)#ないとまえのindexも残る

# pipe()を使ってみた。とても便利。引数はカッコなしで関数名を入れて、df以外の引数を第二引数以降に入れる。
df_73['sales_ymd']=df_73['sales_ymd'].pipe(pd.to_datetime,format='%Y%m%d')
df_73['application_date']=df_73['application_date'].pipe(pd.to_datetime,format='%Y%m%d')

df_73['epoch']=df_73[['sales_ymd','application_date']].apply(lambda x: x[0].timestamp()-x[1].timestamp(),axis=1)
df_73.head()

customer_id  sales_ymd application_date        epoch
0  CS006214000001 2018-11-03       2015-02-01  118454400.0
1  CS006214000001 2017-05-09       2015-02-01   71539200.0
2  CS006214000001 2017-06-08       2015-02-01   74131200.0
3  CS006214000001 2018-10-28       2015-02-01  117936000.0
4  CS006214000001 2019-09-08       2015-02-01  145152000.0

In [276]:
# 模範回答の書き方。.astype(np.int64)/10**9でエポック変換できるらしいが、とてもわかりにくい
df_73['epoch2']=(df_73['sales_ymd'].astype(np.int64)/10**9)-(df_73['application_date'].astype(np.int64)/10**9)
df_73.head()

customer_id  sales_ymd application_date        epoch       epoch2
0  CS006214000001 2018-11-03       2015-02-01  118454400.0  118454400.0
1  CS006214000001 2017-05-09       2015-02-01   71539200.0   71539200.0
2  CS006214000001 2017-06-08       2015-02-01   74131200.0   74131200.0
3  CS006214000001 2018-10-28       2015-02-01  117936000.0  117936000.0
4  CS006214000001 2019-09-08       2015-02-01  145152000.0  145152000.0

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [293]:
# datetime_object.weekday()で曜日が数字で出る（月曜＝０、日曜＝７）
# これを利用して足し引きする
# 注意点：.weekday()はあくまで整数を返すだけなので、datetime_objectと直接計算できない。days = object.weekday()となる。
df_74=df_receipt.copy()
df_74['sales_ymd']=df_74['sales_ymd'].pipe(pd.to_datetime,format='%Y%m%d')
df_74['Monday']=df_74['sales_ymd'].apply(lambda x: x-relativedelta(days=x.weekday()))
df_74['from_Monday']=df_74['sales_ymd']-df_74['Monday']
df_74[['sales_ymd','Monday','from_Monday']].head()

sales_ymd     Monday from_Monday
0 2018-11-03 2018-10-29      5 days
1 2018-11-18 2018-11-12      6 days
2 2017-07-12 2017-07-10      2 days
3 2019-02-05 2019-02-04      1 days
4 2018-08-21 2018-08-20      1 days

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [298]:
df_customer.sample(frac=0.01).head(10)
# 引数
# n=5 ... サンプル数を指定
# frac=0.05 ... 割合で指定
# random_state ... 乱数固定
# axis=1にすると列を抽出（あまり使わない）

customer_id customer_name  gender_cd gender   birth_day  age  \
4409   CS006713000013        吉原 まさみ          1     女性  1941-05-16   77   
21363  CS001314000260        松田 由美子          1     女性  1987-05-18   31   
18413  CS039715000031         川瀬 真帆          1     女性  1944-12-02   74   
11538  CS027503000015         福岡 鉄洋          9     不明  1962-06-20   56   
21147  CS005311000034          玉置 薫          1     女性  1982-02-20   37   
14665  CS006711000013        福士 なつみ          1     女性  1939-05-07   79   
12515  CS010711000027         水口 美優          1     女性  1941-09-14   77   
14586  CS017512000138        田村 小百合          1     女性  1967-01-10   52   
4023   CS033313000080         三上 夏空          1     女性  1980-07-03   38   
5104   CS051401000001          五大 徹          0     男性  1975-08-13   43   

      postal_cd                  address application_store_cd  \
4409   224-0056  神奈川県横浜市都筑区川和台**********               S14006   
21363  144-0055      東京都大田区仲六郷**********               S13001   
18413  167-0032       東京都杉並区天沼**********               S13039   
11538  251-0015      神奈川県藤沢市川名**********               S14027   
21147  177-0045     東京都練馬区石神井台**********               S13005   
14665  224-0034  神奈川県横浜市都筑区勝田町**********               S14006   
12515  223-0057  神奈川県横浜市港北区新羽町**********               S14010   
14586  166-0001     東京都杉並区阿佐谷北**********               S13017   
4023   226-0016   神奈川県横浜市緑区霧が丘**********               S14033   
5104   332-0021      埼玉県川口市西川口**********               S13051   

       application_date     status_cd     age_group  
4409           20151029  0-00000000-0   [60.0, inf)  
21363          20170814  0-00000000-0  [30.0, 40.0)  
18413          20150429  0-00000000-0   [60.0, inf)  
11538          20150509  0-00000000-0  [50.0, 60.0)  
21147          20161105  1-20090522-1  [30.0, 40.0)  
14665          20151105  0-00000000-0   [60.0, inf)  
12515          20160202  0-00000000-0   [60.0, inf)  
14586          20150923  5-20091227-7  [50.0, 60.0)  
4023           20150415  0-00000000-0  [30.0, 40.0)  
5104           20180822  0-00000000-0  [40.0, 50.0)

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [319]:
# 自力回答：

# ↓意味なかったけどいつか使えそうな取り出し方だから残す
frac1, frac0, frac9=df_customer['gender_cd'].value_counts()/df_customer['gender_cd'].size

# 地道な方法
df76_1=df_customer[df_customer['gender_cd']==1].sample(frac=0.1)
df76_0=df_customer[df_customer['gender_cd']==0].sample(frac=0.1)
df76_9=df_customer[df_customer['gender_cd']==9].sample(frac=0.1)
df76=pd.concat([df76_1,df76_0,df76_9])
df76.groupby('gender_cd').count()

customer_id  customer_name  gender  birth_day   age  postal_cd  \
gender_cd                                                                   
0                  298            298     298        298   298        298   
1                 1792           1792    1792       1792  1792       1792   
9                  107            107     107        107   107        107   

           address  application_store_cd  application_date  status_cd  \
gender_cd                                                               
0              298                   298               298        298   
1             1792                  1792              1792       1792   
9              107                   107               107        107   

           age_group  
gender_cd             
0                298  
1               1792  
9                107

In [321]:
# 模範回答：sklearn.model_selection.train_test_splitを使用した例
# train, testが返り値、stratifyで指定したデータは割合がtrainとtestで一定になる
_, df_tmp = train_test_split(df_customer, test_size=0.1, stratify=df_customer['gender'])
df_tmp.groupby('gender_cd').agg({'customer_id' : 'count'})

customer_id
gender_cd             
0                  298
1                 1793
9                  107

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [4]:
# 正直な回答：平均値と標準偏差σを求めてフィルタする。
df_77a=df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id').sum('amount')
amount_mean=df_77a['amount'].mean()
amount_std=df_77a['amount'].std()
df_77a[abs(df_77a['amount'])>=(amount_mean+3*amount_std)]

sales_ymd  sales_epoch  receipt_no  receipt_sub_no  quantity  \
customer_id                                                                    
CS001605000009  363312500  27750988800       20476              27        18   
CS006415000147  564956396  42474412800       32156              42        28   
CS006515000023  605443622  46067788800       34600              45        31   
CS006515000125  524696832  39779424000       29592              39        26   
CS006515000209  403591122  30487276800       22540              30        21   
...                   ...          ...         ...             ...       ...   
CS040214000008  928207758  70010784000        5152              69        46   
CS040414000077  565017834  42703545600       32276              42        29   
CS040415000035  403552570  30402259200       22960              30        20   
CS040415000200  363231068  27459475200       20396              27        18   
CS040515000225  524635422  39553228800       29792              39        26   

                amount  
customer_id             
CS001605000009   18925  
CS006415000147   12723  
CS006515000023   18372  
CS006515000125   12575  
CS006515000209   11373  
...                ...  
CS040214000008   13523  
CS040414000077   11282  
CS040415000035   13119  
CS040415000200   11137  
CS040515000225   14867  

[140 rows x 6 columns]

In [3]:
# 模範回答と同じpreprocessing.scaleを利用する方法。
# スケーリングを調べると平均０分散１という記述が多いが、実際は値は標準偏差にスケールされているようである。
# (分散が１の時、標準偏差は必ず１なので齟齬は出ない）
df_77=df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id').sum('amount')
df_77['amount_scaled']=preprocessing.scale(df_77['amount'])
df_77[abs(df_77['amount_scaled'])>=3]

sales_ymd  sales_epoch  receipt_no  receipt_sub_no  quantity  \
customer_id                                                                    
CS001605000009  363312500  27750988800       20476              27        18   
CS006415000147  564956396  42474412800       32156              42        28   
CS006515000023  605443622  46067788800       34600              45        31   
CS006515000125  524696832  39779424000       29592              39        26   
CS006515000209  403591122  30487276800       22540              30        21   
...                   ...          ...         ...             ...       ...   
CS040214000008  928207758  70010784000        5152              69        46   
CS040414000077  565017834  42703545600       32276              42        29   
CS040415000035  403552570  30402259200       22960              30        20   
CS040415000200  363231068  27459475200       20396              27        18   
CS040515000225  524635422  39553228800       29792              39        26   

                amount  amount_scaled  
customer_id                            
CS001605000009   18925       6.019921  
CS006415000147   12723       3.740202  
CS006515000023   18372       5.816651  
CS006515000125   12575       3.685800  
CS006515000209   11373       3.243972  
...                ...            ...  
CS040214000008   13523       4.034264  
CS040414000077   11282       3.210522  
CS040415000035   13119       3.885763  
CS040415000200   11137       3.157223  
CS040515000225   14867       4.528289  

[140 rows x 7 columns]

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [19]:
# 正直な回答：query内で変数を使う場合は@をつける
df_78=df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id').sum('amount')
q1,q2,q3=df_78['amount'].quantile([0.25,0.50,0.75])
iqr=q3-q1
df_78.query('(@q1-(@iqr*1.5))>amount or amount>(@q3+@iqr*1.5)')

sales_ymd  sales_epoch  receipt_no  receipt_sub_no  quantity  \
customer_id                                                                    
CS001414000048  322811708  24265267200       18612              24        17   
CS001605000009  363312500  27750988800       20476              27        18   
CS002415000594  161445934  12260160000        9136              12         8   
CS004414000181  161427490  12235104000        9256              12         8   
CS005415000137  403548878  30300134400       22940              30        20   
...                   ...          ...         ...             ...       ...   
CS040415000220  322925862  24487315200       18212              24        16   
CS040515000089  645712410  49019904000       36664              48        32   
CS040515000225  524635422  39553228800       29792              39        26   
CS041411000001  605423596  46015603200       34560              45        31   
CS044415000001  444039758  33994080000       25344              33        22   

                amount  
customer_id             
CS001414000048    8584  
CS001605000009   18925  
CS002415000594    9568  
CS004414000181    9584  
CS005415000137    8734  
...                ...  
CS040415000220   10158  
CS040515000089    9240  
CS040515000225   14867  
CS041411000001    9694  
CS044415000001    8556  

[393 rows x 6 columns]

In [23]:
# 最終行のみ：別の書き方
df_78[(df_78['amount']>(q3+(iqr*1.5))) | (df_78['amount']<(q1-(iqr*1.5)))]

sales_ymd  sales_epoch  receipt_no  receipt_sub_no  quantity  \
customer_id                                                                    
CS001414000048  322811708  24265267200       18612              24        17   
CS001605000009  363312500  27750988800       20476              27        18   
CS002415000594  161445934  12260160000        9136              12         8   
CS004414000181  161427490  12235104000        9256              12         8   
CS005415000137  403548878  30300134400       22940              30        20   
...                   ...          ...         ...             ...       ...   
CS040415000220  322925862  24487315200       18212              24        16   
CS040515000089  645712410  49019904000       36664              48        32   
CS040515000225  524635422  39553228800       29792              39        26   
CS041411000001  605423596  46015603200       34560              45        31   
CS044415000001  444039758  33994080000       25344              33        22   

                amount  
customer_id             
CS001414000048    8584  
CS001605000009   18925  
CS002415000594    9568  
CS004414000181    9584  
CS005415000137    8734  
...                ...  
CS040415000220   10158  
CS040515000089    9240  
CS040515000225   14867  
CS041411000001    9694  
CS044415000001    8556  

[393 rows x 6 columns]

In [20]:
# 模範回答：
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")', engine='python'). \
    groupby('customer_id').agg({'amount':'sum'}).reset_index()

pct75 = np.percentile(df_sales_amount['amount'], q=75)
pct25 = np.percentile(df_sales_amount['amount'], q=25)
iqr = pct75 - pct25
amount_low = pct25 - (iqr * 1.5)
amount_hight = pct75 + (iqr * 1.5)
df_sales_amount.query('amount < @amount_low or @amount_hight < amount')

customer_id  amount
98    CS001414000048    8584
332   CS001605000009   18925
549   CS002415000594    9568
1180  CS004414000181    9584
1558  CS005415000137    8734
...              ...     ...
8068  CS040415000220   10158
8126  CS040515000089    9240
8135  CS040515000225   14867
8171  CS041411000001    9694
8241  CS044415000001    8556

[393 rows x 2 columns]

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [25]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

In [26]:
# isnull()用法 https://note.nkmk.me/python-pandas-nan-judge-count/
df_product[df_product['unit_price'].isnull()]

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
159   P040802007                  4                 408              40802   
196   P050103021                  5                 501              50103   
496   P050405009                  5                 504              50405   
1531  P060802026                  6                 608              60802   
2012  P070202092                  7                 702              70202   
6296  P080504027                  8                 805              80504   
7075  P090204185                  9                 902              90204   

      unit_price  unit_cost  
159          NaN        NaN  
196          NaN        NaN  
496          NaN        NaN  
1531         NaN        NaN  
2012         NaN        NaN  
6296         NaN        NaN  
7075         NaN        NaN

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [32]:
df_product1=df_product.dropna()
df_product.count()-df_product1.count()

product_cd            7
category_major_cd     7
category_medium_cd    7
category_small_cd     7
unit_price            0
unit_cost             0
dtype: int64

In [35]:
# sizeは全項目の数だから少し違う
df_product.size-df_product1.size

42

In [36]:
# 行数はlen(df)が正解
len(df_product)-len(df_product1)

7

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [45]:
# 自力回答：正直に実装だが手間がかかっている
mean_unit_price=df_product['unit_price'].dropna().mean()
mean_unit_cost=df_product['unit_cost'].dropna().mean()
df_product2=df_product.copy()
print(df_product2['unit_price'].isnull().sum())
print(df_product2['unit_cost'].isnull().sum())
df_product2[df_product2['unit_price'].isnull()]=mean_unit_price
df_product2[df_product2['unit_cost'].isnull()]=mean_unit_cost
print(df_product2['unit_price'].isnull().sum())
print(df_product2['unit_cost'].isnull().sum())


7
7
0
0


In [46]:
# 模範回答：fillna()メソッドとnp.nanmean()を使ってより完結に実装。可読性も高い。
df_product_2 = df_product.fillna({'unit_price':np.round(np.nanmean(df_product['unit_price'])), 
                                  'unit_cost':np.round(np.nanmean(df_product['unit_cost']))})
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [50]:
df_product_3=df_product.copy()
df_product_3=df_product_3.fillna({'unit_price':np.round(np.nanmedian(df_product_3['unit_price'])),\
                     'unit_cost':np.round(np.nanmedian(df_product_3['unit_cost']))})
df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# 自力回答：結構無理矢理実装（？）検証しながら

In [70]:
# category_small_cdの中央値を算出し、参照用dfを作成
df_83=np.round(df_product.groupby('category_small_cd').median())
df_83

category_major_cd  category_medium_cd  unit_price  \
category_small_cd                                                      
40101                              4                 401       283.0   
40102                              4                 401       378.0   
40201                              4                 402       223.0   
40202                              4                 402       178.0   
40203                              4                 402       308.0   
...                              ...                 ...         ...   
91401                              9                 914       315.0   
91402                              9                 914       900.0   
91403                              9                 914       652.0   
91501                              9                 915       510.0   
91503                              9                 915      1080.0   

                   unit_cost  
category_small_cd             
40101                  212.0  
40102                  284.0  
40201                  168.0  
40202                  134.0  
40203                  231.0  
...                      ...  
91401                  236.0  
91402                  675.0  
91403                  490.0  
91501                  383.0  
91503                  810.0  

[228 rows x 4 columns]

In [92]:
# 検証用：.applyで処理する動きを作るためにrowを取り出したseriesを準備
test83=df_product.iloc[1]
test83

product_cd            P040101002
category_major_cd              4
category_medium_cd           401
category_small_cd          40101
unit_price                   218
unit_cost                    164
Name: 1, dtype: object

In [78]:
# row_seriesを使って参照用dfから値を抽出する検証
df_83.at[test83['category_small_cd'],'unit_price']

283.0

In [85]:
# row_seriesのunit_priceがnaかどうかを判定したい
# 下の書き方だとunit_priceを取り出した時点でseriesではないため.isnull()メソッドが使えないことが判明
# この場合はnumpy.float64に合わせて〜というのでも動くと思うが、毎度データ型を機にする必要が出てきてしまうためseriesで扱いisnull()を使いたい
test83['unit_price'].isnull()

AttributeError: 'numpy.float64' object has no attribute 'isnull'

In [86]:
# 単純にseriesの状態でisnull()をしてからunit_price項目を取り出せば良い
test83.isnull()['unit_price']

False

In [91]:
# 実装：上記の素材と検証をもとに
df_83=np.round(df_product.groupby('category_small_cd').median())
df_product_4=df_product.copy()
df_product_4['unit_price2']=df_product_4.apply(lambda x: df_83.at[x['category_small_cd'],'unit_price'] \
                                               if x.isnull()['unit_price']==True else x['unit_price'],axis=1)
df_product_4['unit_cost2']=df_product_4.apply(lambda x: df_83.at[x['category_small_cd'],'unit_cost'] \
                                              if x.isnull()['unit_cost']==True else x['unit_cost'],axis=1)
df_product_4.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
unit_price2           0
unit_cost2            0
dtype: int64

In [93]:
# 模範回答：結局似たような実装法であった。np.isnan()でnumpy型データの判定もできるらしい。
df_tmp = df_product.groupby('category_small_cd').agg({'unit_price':'median', 'unit_cost':'median'}).reset_index()
df_tmp.columns = ['category_small_cd', 'median_price', 'median_cost']

df_product_4 = pd.merge(df_product, df_tmp, how='inner', on='category_small_cd')

df_product_4['unit_price'] = df_product_4[['unit_price', 'median_price']]. \
                    apply(lambda x: np.round(x[1]) if np.isnan(x[0]) else x[0], axis=1)
df_product_4['unit_cost'] = df_product_4[['unit_cost', 'median_cost']]. \
                    apply(lambda x: np.round(x[1]) if np.isnan(x[0]) else x[0], axis=1)

df_product_4.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
median_price          0
median_cost           0
dtype: int64

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [96]:
df_84=df_customer.pipe(pd.merge,df_receipt,on='customer_id',how='outer')
df_84.head(3)

customer_id customer_name  gender_cd gender   birth_day   age postal_cd  \
0  CS021313000114        大野 あや子        1.0     女性  1981-04-29  37.0  259-1113   
1  CS037613000071         六角 雅彦        9.0     不明  1952-04-01  66.0  136-0076   
2  CS031415000172       宇多田 貴美子        1.0     女性  1976-10-04  42.0  151-0053   

                address application_store_cd  application_date     status_cd  \
0  神奈川県伊勢原市粟窪**********               S14021        20150905.0  0-00000000-0   
1    東京都江東区南砂**********               S13037        20150414.0  0-00000000-0   
2   東京都渋谷区代々木**********               S13031        20150529.0  D-20100325-C   

    sales_ymd   sales_epoch store_cd  receipt_no  receipt_sub_no  product_cd  \
0         NaN           NaN      NaN         NaN             NaN         NaN   
1         NaN           NaN      NaN         NaN             NaN         NaN   
2  20170507.0  1.494115e+09   S13031      1102.0             1.0  P060103001   

   quantity  amount  
0       NaN     NaN  
1       NaN     NaN  
2       1.0   100.0

In [100]:
df_84a=df_84[['customer_id','amount']].groupby('customer_id').sum()
df_84a.sort_values('amount',ascending=False).head(3)

amount
customer_id               
ZZ000000000000  12395003.0
CS017415000097     23086.0
CS015415000185     20153.0

In [120]:
df_84['sales_ymd']=pd.to_datetime(df_84['sales_ymd'],format='%Y%m%d')
df_84['sales_year']=df_84['sales_ymd'].apply(lambda x: x.year)
df_84b=df_84.query('sales_year==2019')[['customer_id','amount']].groupby('customer_id').sum()
df_84b.sort_values('amount',ascending=False).head(3)

amount
customer_id              
ZZ000000000000  4411368.0
CS001605000009    13983.0
CS011415000006    13404.0

In [123]:
df_84c=pd.merge(df_84a,df_84b,how='outer',on='customer_id')
df_84c.sort_values('amount_x',ascending=False).head(3)

amount_x   amount_y
customer_id                          
ZZ000000000000  12395003.0  4411368.0
CS017415000097     23086.0     2564.0
CS015415000185     20153.0     4095.0

In [126]:
df_84c['2019_rate']=df_84c['amount_y']/df_84c['amount_x']
df_84c.sort_values('amount_x',ascending=False).head(3)

amount_x   amount_y  2019_rate
customer_id                                     
ZZ000000000000  12395003.0  4411368.0   0.355899
CS017415000097     23086.0     2564.0   0.111063
CS015415000185     20153.0     4095.0   0.203196

In [129]:
print(df_84c.isnull().sum())
print(df_84c.fillna(0).isnull().sum())
df_84c=df_84c.fillna(0)

amount_x         0
amount_y     16952
2019_rate    16952
dtype: int64
amount_x     0
amount_y     0
2019_rate    0
dtype: int64


In [130]:
df_84c[df_84c['2019_rate']>0].head()

amount_x  amount_y  2019_rate
customer_id                                  
CS001113000004    1298.0    1298.0   1.000000
CS001114000005     626.0     188.0   0.300319
CS001115000010    3044.0     578.0   0.189882
CS001205000004    1988.0     702.0   0.353119
CS001205000006    3337.0     486.0   0.145640

In [132]:
# 模範回答：
# sales_ymdがfloatなので、数字でフィルタしている。
# 後は手順や細かい方法は違うが基本的には同じようなやり方
df_tmp_1 = df_receipt.query('20190101 <= sales_ymd <= 20191231')
df_tmp_1 = pd.merge(df_customer['customer_id'], df_tmp_1[['customer_id', 'amount']], how='left', on='customer_id'). \
    groupby('customer_id').sum().reset_index().rename(columns={'amount':'amount_2019'})

df_tmp_2 = pd.merge(df_customer['customer_id'], df_receipt[['customer_id', 'amount']], how='left', on='customer_id'). \
    groupby('customer_id').sum().reset_index()

df_tmp = pd.merge(df_tmp_1, df_tmp_2, how='inner', on='customer_id')
df_tmp['amount_rate'] = df_tmp['amount_2019'] / df_tmp['amount']

df_tmp.query('amount_rate > 0').head(10)

customer_id  amount_2019  amount  amount_rate
8   CS001113000004       1298.0  1298.0     1.000000
10  CS001114000005        188.0   626.0     0.300319
12  CS001115000010        578.0  3044.0     0.189882
17  CS001205000004        702.0  1988.0     0.353119
18  CS001205000006        486.0  3337.0     0.145640
23  CS001211000025        456.0   456.0     1.000000
30  CS001212000070        456.0   456.0     1.000000
57  CS001214000009        664.0  4685.0     0.141729
59  CS001214000017       2962.0  4132.0     0.716844
61  CS001214000048       1889.0  2374.0     0.795703

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [136]:
df_85=df_geocode[['postal_cd','longitude','latitude']].groupby('postal_cd').mean()
df_customer_1=pd.merge(df_customer[['customer_id','postal_cd']], df_85, on='postal_cd', how='inner')
df_customer_1.head(10)

customer_id postal_cd  longitude  latitude
0  CS021313000114  259-1113  139.31779  35.41358
1  CS021303000023  259-1113  139.31779  35.41358
2  CS021303000007  259-1113  139.31779  35.41358
3  CS021313000183  259-1113  139.31779  35.41358
4  CS021314000098  259-1113  139.31779  35.41358
5  CS021314000093  259-1113  139.31779  35.41358
6  CS021413000049  259-1113  139.31779  35.41358
7  CS037613000071  136-0076  139.83502  35.67193
8  CS015415000209  136-0076  139.83502  35.67193
9  CS037614000045  136-0076  139.83502  35.67193

In [137]:
# 模範回答：
df_customer_1 = pd.merge(df_customer[['customer_id', 'postal_cd']],
                         df_geocode[['postal_cd', 'longitude' ,'latitude']],
                         how='inner', on='postal_cd')
df_customer_1 = df_customer_1.groupby('customer_id'). \
    agg({'longitude':'mean', 'latitude':'mean'}).reset_index(). \
    rename(columns={'longitude':'m_longitude', 'latitude':'m_latitude'})

df_customer_1 = pd.merge(df_customer, df_customer_1, how='inner', on='customer_id')
df_customer_1.head(3)

customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1     女性  1981-04-29   37  259-1113   
1  CS037613000071         六角 雅彦          9     不明  1952-04-01   66  136-0076   
2  CS031415000172       宇多田 貴美子          1     女性  1976-10-04   42  151-0053   

                address application_store_cd  application_date     status_cd  \
0  神奈川県伊勢原市粟窪**********               S14021          20150905  0-00000000-0   
1    東京都江東区南砂**********               S13037          20150414  0-00000000-0   
2   東京都渋谷区代々木**********               S13031          20150529  D-20100325-C   

   m_longitude  m_latitude  
0    139.31779    35.41358  
1    139.83502    35.67193  
2    139.68965    35.67374

---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [139]:
df_86=pd.merge(df_customer_1,df_store,left_on='application_store_cd',right_on='store_cd',how='inner').drop('store_cd',axis=1)
df_86.head()
# 三角関数部分は模範回答参照。arccosはcosの逆関数のことらしい。逆関数は y = cos(x) のとき x = arccos(y) ということ　らしい。

customer_id customer_name  gender_cd gender   birth_day  age postal_cd  \
0  CS021313000114        大野 あや子          1     女性  1981-04-29   37  259-1113   
1  CS021313000025        砂川 あさみ          1     女性  1981-10-05   37  259-1131   
2  CS021411000096          布施 杏          1     女性  1971-10-30   47  259-1114   
3  CS021415000150         早美 紗季          1     女性  1969-11-26   49  259-1141   
4  CS021313000046          会田 薫          9     不明  1983-01-19   36  259-1144   

               address_x application_store_cd  application_date  ...  \
0   神奈川県伊勢原市粟窪**********               S14021          20150905  ...   
1  神奈川県伊勢原市伊勢原**********               S14021          20150326  ...   
2   神奈川県伊勢原市高森**********               S14021          20160621  ...   
3  神奈川県伊勢原市上粕屋**********               S14021          20150601  ...   
4   神奈川県伊勢原市池端**********               S14021          20150722  ...   

  m_latitude  store_name  prefecture_cd prefecture       address_y  \
0   35.41358        伊勢原店             14       神奈川県  神奈川県伊勢原市伊勢原四丁目   
1   35.39748        伊勢原店             14       神奈川県  神奈川県伊勢原市伊勢原四丁目   
2   35.42174        伊勢原店             14       神奈川県  神奈川県伊勢原市伊勢原四丁目   
3   35.41649        伊勢原店             14       神奈川県  神奈川県伊勢原市伊勢原四丁目   
4   35.39936        伊勢原店             14       神奈川県  神奈川県伊勢原市伊勢原四丁目   

            address_kana        tel_no longitude  latitude  floor_area  
0  カナガワケンイセハラシイセハラヨンチョウメ  046-123-4035  139.3129  35.40169       962.0  
1  カナガワケンイセハラシイセハラヨンチョウメ  046-123-4035  139.3129  35.40169       962.0  
2  カナガワケンイセハラシイセハラヨンチョウメ  046-123-4035  139.3129  35.40169       962.0  
3  カナガワケンイセハラシイセハラヨンチョウメ  046-123-4035  139.3129  35.40169       962.0  
4  カナガワケンイセハラシイセハラヨンチョウメ  046-123-4035  139.3129  35.40169       962.0  

[5 rows x 22 columns]

In [ ]:
# 模範回答：三角関数を使いこなす必要
def calc_distance(x1, y1, x2, y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1)) * math.sin(math.radians(y2)) 
                       + math.cos(math.radians(y1)) * math.cos(math.radians(y2)) 
                            * math.cos(math.radians(x1) - math.radians(x2)))
    return distance

df_tmp = pd.merge(df_customer_1, df_store, how='inner', left_on='application_store_cd', right_on='store_cd')    

df_tmp['distance'] =   df_tmp[['m_longitude', 'm_latitude','longitude', 'latitude']]. \
                                apply(lambda x: calc_distance(x[0], x[1], x[2], x[3]), axis=1)

df_tmp[['customer_id', 'address_x', 'address_y', 'distance']].head(10)

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [143]:
# 模範回答：sort_valuesとdrop_duplicatesのオプションの組み合わせで実装されている（自力でできなかった）
df_tmp = df_receipt.groupby('customer_id').agg({'amount':sum}).reset_index()
df_customer_u = pd.merge(df_customer, df_tmp, how='left', on='customer_id').sort_values(['amount', 'customer_id']
                                                                                        , ascending=[False, True])
df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'], keep='first', inplace=True)

print('減少数: ', len(df_customer) - len(df_customer_u))

減少数:  30


---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [ ]:
# 模範回答：ほぼ前問で達成できていることなので飛ばした
df_customer_n = pd.merge(df_customer, df_customer_u[['customer_name', 'postal_cd', 'customer_id']],
                        how='inner', on =['customer_name', 'postal_cd'])
df_customer_n.rename(columns={'customer_id_x':'customer_id', 'customer_id_y':'integration_id'}, inplace=True)

print('ID数の差', len(df_customer_n['customer_id'].unique()) - len(df_customer_n['integration_id'].unique()))

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [155]:
df_89=pd.merge(df_customer,df_receipt,how='inner',on='customer_id')
print(len(df_89), df_89['customer_id'].nunique())
df_89=df_89.drop_duplicates('customer_id')
print(len(df_89), df_89['customer_id'].nunique())

df_train, df_test = train_test_split(df_89, test_size=0.2)
print(len(df_train),len(df_test),len(df_test)/(len(df_train)+len(df_test)))

65682 8306
8306 8306
6644 1662 0.20009631591620516


In [156]:
# 模範回答：drop_duplicates()してないからreceiptデータ数のcustomer_idがあるけどいいのだろうか？
df_tmp = pd.merge(df_customer, df_receipt['customer_id'], how='inner', on='customer_id')
df_train, df_test = train_test_split(df_tmp, test_size=0.2, random_state=71)
print('学習データ割合: ', len(df_train) / len(df_tmp))
print('テストデータ割合: ', len(df_test) / len(df_tmp))

学習データ割合:  0.7999908650771901
テストデータ割合:  0.2000091349228099
65682


---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [183]:
# 解釈を加える：各学習用、テスト用のデータを被り無しでランダムに分割する方法を考えた。
df_90=df_receipt.copy()
df_90['sales_ymd']=df_90['sales_ymd'].pipe(pd.to_datetime,format='%Y%m%d')
df_90['sales_year']=df_90['sales_ymd'].apply(lambda x: x.year)
df_90['sales_month']=df_90['sales_ymd'].apply(lambda x: x.month)

df_90a=df_90.groupby(['sales_year','sales_month']).sum('amount')
print(len(df_90a))# 34ヶ月分のデータがある。

# train_test_split()でデータを取り出した後、元のデータからそれらをdropする。
train90_1, test90_1 = train_test_split(df_90a, train_size=6, test_size=3)
df_90a=df_90a.drop(train90_1.index)
df_90a=df_90a.drop(test90_1.index)
# そのデータからさらにデータを取り出すことで被り無しにできる。
train90_2, test90_2 = train_test_split(df_90a, train_size=6, test_size=3)
df_90a=df_90a.drop(train90_2.index)
df_90a=df_90a.drop(test90_2.index)

train90_3, test90_3 = train_test_split(df_90a, train_size=6, test_size=3)
df_90a=df_90a.drop(train90_3.index)
df_90a=df_90a.drop(test90_3.index)

print(len(df_90a))

34
7


In [176]:
# 模範回答：被り有りだしランダム性がない。普通にランダム抽出を３回やったほうがバイアスが無い気がする。
df_tmp = df_receipt[['sales_ymd', 'amount']].copy()
df_tmp['sales_ym'] = df_tmp['sales_ymd'].astype('str').str[0:6]
df_tmp = df_tmp.groupby('sales_ym').agg({'amount':'sum'}).reset_index()

# 関数化することで長期間データに対する多数のデータセットもループなどで処理できるようにする
def split_data(df, train_size, test_size, slide_window, start_point):
    train_start = start_point * slide_window
    test_start = train_start + train_size
    return df[train_start : test_start], df[test_start : test_start + test_size]

df_train_1, df_test_1 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=0)
df_train_2, df_test_2 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=1)
df_train_3, df_test_3 = split_data(df_tmp, train_size=12, test_size=6, slide_window=6, start_point=2)

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [209]:
# 無理矢理気味。売上実績の有無でラベルを作成し、別々に同数を取得して結合
df_91=df_receipt.groupby('customer_id').sum('amount')

df_91=pd.merge(df_customer,df_91,on='customer_id',how='outer')
df_91['bought']=df_91['amount'].apply(lambda x: 1 if x>0 else 0)
print(df_91['bought'].value_counts())

df_91a=pd.concat([df_91[df_91['bought']==0].sample(min(df_91['bought'].value_counts())), \
                  df_91[df_91['bought']==1].sample(min(df_91['bought'].value_counts()))])
df_91a['bought'].value_counts()

0    13665
1     8307
Name: bought, dtype: int64


1    8307
0    8307
Name: bought, dtype: int64

In [204]:
# 模範回答：

# pd.mergeでhow='left'にしている　→　より明確で良い
# lamdaも同じような感じでラベル付けしている
# imbalanced-learnは不均衡データの解析を助けるライブラる
# imblearn.under_sampling.RandomUnderSamplerはクラスである
# インスタンスを作成し、.fit_sample(data1,date2)とすることでリサイズされたdata1とdata2を取得。サイズは小さい方に合わせる形になる。

#unbalancedのubUnderを使った例
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_tmp = pd.merge(df_customer, df_tmp, how='left', on='customer_id')
df_tmp['buy_flg'] = df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)

print('0の件数', len(df_tmp.query('buy_flg == 0')))
print('1の件数', len(df_tmp.query('buy_flg == 1')))

positive_count = len(df_tmp.query('buy_flg == 1'))

rs = RandomUnderSampler(random_state=71)

df_sample, _ = rs.fit_sample(df_tmp, df_tmp.buy_flg)

print('0の件数', len(df_sample.query('buy_flg == 0')))
print('1の件数', len(df_sample.query('buy_flg == 1')))

0の件数 13665
1の件数 8306
0の件数 8306
1の件数 8306


In [ ]:
# Oversampling：少数側のデータ数を水増しして多数側との差を埋める
# Undersampling：多数側のデータ数を減らして少数側と合わせる
# https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.under_sampling.RandomUnderSampler.html

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [219]:
# DBの正規化：ググると色々出てくるが、テーブルをなるべく分離して管理しやすくするイメージ（必要な時に繋げれば良い）
# https://breezegroup.co.jp/202005/database-normalization/

df_92=df_customer.drop('gender',axis=1)
df_gender=df_customer[['gender_cd','gender']].drop_duplicates().reset_index(drop=True)
df_gender

gender_cd gender
0          1     女性
1          9     不明
2          0     男性

In [220]:
# 模範回答：同じ
df_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()
df_customer_s = df_customer.drop(columns='gender')

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [231]:
df_93=\
df_product.pipe(pd.merge,df_category[['category_major_cd','category_major_name']],on='category_major_cd',how='left')\
.pipe(pd.merge,df_category[['category_medium_cd','category_medium_name']],on='category_medium_cd',how='left')\
.pipe(pd.merge,df_category[['category_small_cd','category_small_name']],on='category_small_cd',how='left')
# これだとカラム順が見難いため非正規化した意味が半減　→ 順番を変える

In [247]:
# 順番の変え方としてdf.reindex([col1,col2,col3...])をするのが基本だが、数が多いので少し工夫する。
col_93=df_93.columns.to_list()# =list(df_93.columns) でも同じ。
print(col_93, len(col_93))
inss=col_93[6:]
print(inss)
col_93.insert(2,inss[0])
col_93.insert(4,inss[1])
col_93.insert(6,inss[2])
col_93_new=col_93[:9]
print(col_93_new)
#col_93.insert(2,inss[0]).insert(4,inss[1]).insert(6,inss[2])

['product_cd', 'category_major_cd', 'category_medium_cd', 'category_small_cd', 'unit_price', 'unit_cost', 'category_major_name', 'category_medium_name', 'category_small_name'] 9
['category_major_name', 'category_medium_name', 'category_small_name']
['product_cd', 'category_major_cd', 'category_major_name', 'category_medium_cd', 'category_medium_name', 'category_small_cd', 'category_small_name', 'unit_price', 'unit_cost']


In [249]:
# 別の並び替え方: いっそシンプルに名前順にする
col_93a=df_93.columns.to_list()
col_93a.sort()
col_93a

['category_major_cd',
 'category_major_name',
 'category_medium_cd',
 'category_medium_name',
 'category_small_cd',
 'category_small_name',
 'product_cd',
 'unit_cost',
 'unit_price']

In [255]:
df_93_fin=df_93.reindex(col_93_new, axis=1)
df_93_fin.head()

product_cd  category_major_cd category_major_name  category_medium_cd  \
0  P040101001                  4                  惣菜                 401   
1  P040101001                  4                  惣菜                 401   
2  P040101001                  4                  惣菜                 401   
3  P040101001                  4                  惣菜                 401   
4  P040101001                  4                  惣菜                 401   

  category_medium_name  category_small_cd category_small_name  unit_price  \
0                  御飯類              40101                 弁当類       198.0   
1                  御飯類              40101                 弁当類       198.0   
2                  御飯類              40101                 弁当類       198.0   
3                  御飯類              40101                 弁当類       198.0   
4                  御飯類              40101                 弁当類       198.0   

   unit_cost  
0      149.0  
1      149.0  
2      149.0  
3      149.0  
4      149.0

In [257]:
# 模範回答：並び替えまでしていない
df_product_full = pd.merge(df_product, df_category[['category_small_cd', 
                                                    'category_major_name',
                                                    'category_medium_name',
                                                    'category_small_name']], 
                           how = 'inner', on = 'category_small_cd')
df_product_full.head()

product_cd  category_major_cd  category_medium_cd  category_small_cd  \
0  P040101001                  4                 401              40101   
1  P040101002                  4                 401              40101   
2  P040101003                  4                 401              40101   
3  P040101004                  4                 401              40101   
4  P040101005                  4                 401              40101   

   unit_price  unit_cost category_major_name category_medium_name  \
0       198.0      149.0                  惣菜                  御飯類   
1       218.0      164.0                  惣菜                  御飯類   
2       230.0      173.0                  惣菜                  御飯類   
3       248.0      186.0                  惣菜                  御飯類   
4       268.0      201.0                  惣菜                  御飯類   

  category_small_name  
0                 弁当類  
1                 弁当類  
2                 弁当類  
3                 弁当類  
4                 弁当類

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [256]:
export_directory='/Users/yuya/pipenv/test36/100knock/export'

# to_csv https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html
df_93_fin.to_csv(f'{export_directory}/csv93.csv', header=True, encoding='utf-8')
# この二つのオプションはデフォルト値だが一応記述した

In [ ]:
# 模範回答：index=Falseにしてた（デフォルトはTrue）
# フォルダ作るから実行しない
os.makedirs('./data')
df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.csv', encoding='UTF-8', index=False)

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [258]:
df_93_fin.to_csv(f'{export_directory}/csv95.csv', header=True, encoding='cp932')

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [264]:

df_93_fin.to_csv(f'{export_directory}/csv96.csv', header=False, encoding='utf-8')

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [263]:
# P-94で出力したcsvを読み込む
df_97=pd.read_csv(f'{export_directory}/csv93.csv')
df_97.head()
# index列が読み込まれて二重になる。index_col=オプション等で読み込み時に弾くことも可能だが、読んでから外せば問題ない

Unnamed: 0  product_cd  category_major_cd category_major_name  \
0           0  P040101001                  4                  惣菜   
1           1  P040101001                  4                  惣菜   
2           2  P040101001                  4                  惣菜   
3           3  P040101001                  4                  惣菜   
4           4  P040101001                  4                  惣菜   

   category_medium_cd category_medium_name  category_small_cd  \
0                 401                  御飯類              40101   
1                 401                  御飯類              40101   
2                 401                  御飯類              40101   
3                 401                  御飯類              40101   
4                 401                  御飯類              40101   

  category_small_name  unit_price  unit_cost  
0                 弁当類       198.0      149.0  
1                 弁当類       198.0      149.0  
2                 弁当類       198.0      149.0  
3                 弁当類       198.0      149.0  
4                 弁当類       198.0      149.0

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [267]:
# headerがないことを教えてあげないと一行目がheaderになる
df_98=pd.read_csv(f'{export_directory}/csv96.csv', header=None)
df_98.head()

0           1  2   3    4    5      6    7      8      9
0  0  P040101001  4  惣菜  401  御飯類  40101  弁当類  198.0  149.0
1  1  P040101001  4  惣菜  401  御飯類  40101  弁当類  198.0  149.0
2  2  P040101001  4  惣菜  401  御飯類  40101  弁当類  198.0  149.0
3  3  P040101001  4  惣菜  401  御飯類  40101  弁当類  198.0  149.0
4  4  P040101001  4  惣菜  401  御飯類  40101  弁当類  198.0  149.0

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [268]:
# to_csvだけどtsvもこれで出力する。この場合は拡張子を変えて、sep=オプションで調整
df_93_fin.to_csv(f'{export_directory}/csv99.tsv', sep='\t', header=True, encoding='utf-8')

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [269]:
df_100=pd.read_csv(f'{export_directory}/csv99.tsv', sep='\t')
df_100.head()

Unnamed: 0  product_cd  category_major_cd category_major_name  \
0           0  P040101001                  4                  惣菜   
1           1  P040101001                  4                  惣菜   
2           2  P040101001                  4                  惣菜   
3           3  P040101001                  4                  惣菜   
4           4  P040101001                  4                  惣菜   

   category_medium_cd category_medium_name  category_small_cd  \
0                 401                  御飯類              40101   
1                 401                  御飯類              40101   
2                 401                  御飯類              40101   
3                 401                  御飯類              40101   
4                 401                  御飯類              40101   

  category_small_name  unit_price  unit_cost  
0                 弁当類       198.0      149.0  
1                 弁当類       198.0      149.0  
2                 弁当類       198.0      149.0  
3                 弁当類       198.0      149.0  
4                 弁当類       198.0      149.0

# これで１００本終わりです。おつかれさまでした！